<a href="https://colab.research.google.com/github/calm-ixia/SDManualGUI/blob/main/SDManualGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ライブラリ導入

In [ ]:
# Stable Diffusion パッケージのインストール 
# diffusersの公式リポジトリを参照 https://github.com/huggingface/diffusers
# アカウント登録込みの作業フローは https://programmingforever.hatenablog.com/entry/2022/09/21/174953 も参考
!pip install --upgrade diffusers transformers scipy ftfy
!pip install accelerate

#version 7.7.1 < latest 8.0.4
from ipywidgets import widgets, Layout
from PIL import Image, ImageChops
from PIL.PngImagePlugin import PngInfo
import io

import torch
import diffusers
from diffusers import StableDiffusionPipeline, KarrasVePipeline, ScoreSdeVePipeline
from diffusers.schedulers import *
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker

import os
import math
import time

# xformersには未対応
"""
%pip install xformers==0.0.16rc425
# https://note.com/npaka/n/n5fd3d8ecf1e6
# %pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# http://cedro3.com/ai/dream-booth/
#%pip install -q https://github.com/OMGhozlan/xformers_colab/releases/download/1.0.0/xformers-0.0.15.dev0+cu11.2-cp38-cp38-linux_x86_64.whl
"""

"""
Layered Diffusion (七師氏のStable Diffusion改造)
  https://github.com/nanashi161382/unstable_diffusion
  https://note.com/tomo161382/n/nff2aa749c48f
"""
!wget 'https://raw.githubusercontent.com/nanashi161382/unstable_diffusion/main/pipeline_layered_diffusion.py' -O /content/layered_diffusion.py

import layered_diffusion


# Long Prompt Weighting Stable Diffusion (SkyTNT氏) 
# https://github.com/huggingface/diffusers/tree/main/examples/community#long-prompt-weighting-stable-diffusion
# community pipeline から オリジナルの lpw_stable_diffusion を直接利用する方法に変更した。以下の記述は参考として残す

#if using_lpw_stable_diffusion_mod:
#  # huggingfaceのAlanB氏のリポジトリにある、StableDiffusionLongPromptWeightingPipeline をインポート
#  # https://huggingface.co/docs/hub/models-downloading
#  %pip install huggingface_hub
#  from huggingface_hub import hf_hub_url, hf_hub_download
#  REPO_ID = "AlanB/lpw_stable_diffusion_mod"
#  FILENAME = "pipeline.py"
#  cache = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
#!ln -s {cache} lpw_stable_diffusion_mod.py

!wget 'https://raw.githubusercontent.com/huggingface/diffusers/main/examples/community/lpw_stable_diffusion.py' -O /content/lpw_stable_diffusion.py

import lpw_stable_diffusion

# Extras用のルートディレクトリ
HighRes_root = "/content/Real-ESRGAN"
DepthMap_root = "/content/MiDaS"


# （必要時）割り当てられたGPUの確認

In [ ]:
#@markdown 割り当てられたGPUの確認
!nvidia-smi

#@markdown Googleドライブのマウント
"""
from google.colab import drive
drive.mount('/content/drive/')
"""

# （必要時）Hugging Face へのログイン
Stable Diffusion 2.0, 2.1 では不要。runwayml-v1.5モデルの利用には利用同意とHugging Faceへのログインが要

In [ ]:
#@title 利用同意が必要なモデルを利用する際に、Hugging Face に手動でログイン
# その際に、アクセストークンをnotebookに保存せずに起動させるため、手動ログインを採用
#@markdown Hugging Faceへのログインとは別に、利用モデルの利用規約を読んで同意(Agree and access repository)すること。
#@markdown https://zenn.dev/razokulover/scraps/b4f639228ab305

!huggingface-cli login


# （必要時）高解像度化ライブラリReal-ESRGANの導入

In [ ]:
# 公式のReadmeよりコピー

# Clone repo
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd {HighRes_root}

# Install dependent packages
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# facexlib and gfpgan are for face enhancement
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

import argparse
import cv2
import glob
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url

from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact

%cd /content


# （必要時）深度推定ライブラリMiDaSの導入



In [ ]:

!git clone https://github.com/isl-org/MiDaS.git
%cd {DepthMap_root}

# requirements.yaml を基にpipでインストール
!pip install cudatoolkit
#!pip install pytorch
!pip install torchvision
#!pip install numpy
!pip install opencv-python
!pip install imutils
!pip install timm
!pip install einops

import os
import glob
import torch
import utils
import cv2
import argparse
import time

import numpy as np

from imutils.video import VideoStream
import midas
from midas.model_loader import default_models, load_model

#from . import run
from run import process, create_side_by_side

%cd /content


#2. GUI実装

In [ ]:
#@title ユーティリティ

# 共通の処理

# バッチ1回で生成した個々の画像オブジェクトと、バッチ1回分のgrid画像オブジェクトを生成
def create_batch_image(outImages, width, height, grid_column_count):
  #gridを生成
  image_count = len(outImages)
  grid_whole_width = width * grid_column_count
  # バッチ1回ごとにグリッドの列数×行数の矩形を作る。余りが出る場合は黒塗りになる
  grid_whole_height = height * math.ceil(image_count/grid_column_count)
  grid_image = Image.new('RGB', size=(grid_whole_width, grid_whole_height) )

  #画像ファイルを保存
  # バッチ1回での行数
  batch_rows = 1 + ((image_count-1)//grid_column_count)
  for batch_index,image in enumerate(outImages):
    # gridの所定位置にペースト
    paste_col_index = batch_index % grid_column_count
    paste_row_index = batch_index//grid_column_count
    grid_image.paste(image, box=(width*paste_col_index, height*paste_row_index) )
  
  return grid_image

# 画像をファイルに保存
def save_image(image, fp, parameter_text):
  # 画像のタグ情報を生成
  grid_info = PngInfo()
  grid_info.add_text("parameters", parameter_text)
  # gridをファイルに保存
  image.save(fp, format="png", pnginfo=grid_info)

# サムネイルを作成、画像を縮小
def create_thumbnail(image, resize=0.125):
  return image.resize(size=(int(image.width*resize),int(image.height*resize)) )

"""
ipywidgets version 7.7.1の場合、
widgets.FileUpload から得られる値は、
{filename: {"metadata":{}, "content":rawdata} } の構造を持つdict
"""
def update_pnginfo(new_value):
  # ipywidgets version 7.7.1での記述
  for filename, value in new_value.items():
    #metadata = value["metadata"]

    #rawデータを取得
    data = value["content"]
    #画像情報取得用にPILのライブラリで画像を開く
    img = Image.open(io.BytesIO(data))
    #画像情報を取得
    parameter_text = img.info["parameters"]
    return (data, parameter_text)

def fileUploader_load_png(new_value):
  for filename, value in new_value.items():
    #print(value)
    #print(list(value.keys()))
    #metadata = value["metadata"]
    #print(metadata)

    #rawデータを取得(bytesオブジェクト)
    data = value["content"]
    return data

"""
TODO: widgets.FileUploadは ver.8でデータ構造が変更されているため、
Colabのipywidgetsのバージョンがver.8以上になった場合はコードを変更すること。
https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#File-Upload

ver.8 では、valueの値はファイルごとのdictのタプルとなる
uploader.value = (
   {
     'name': 'example.txt',
     'type': 'text/plain',
     'size': 36,
     'last_modified': datetime.datetime(2020, 1, 9, 15, 58, 43, 321000, tzinfo=datetime.timezone.utc),
     'content': <memory at 0x10c1b37c8> # raw data, python 'memoryview' object
   },
   ...
)
"""
pass


In [ ]:
#@title Long Prompt Weighting とマスク操作

# ロード済みのLong Prompt Weighting Pipelineを用いてtext embeddingを生成
# Pipelineの_encode_prompt()だけ使用

from typing import Callable, List, Optional, Union
from diffusers import DiffusionPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput
from diffusers.models import AutoencoderKL, UNet2DConditionModel
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer
import PIL

class LPWEncoding(layered_diffusion.StandardEncoding):
  def __init__(self,
    lpw_pipe, #あらかじめモデルをロードしておく
    num_images_per_prompt: Optional[int] = 1,
    max_embeddings_multiples: Optional[int] = 3
  ):
    self.pipe = lpw_pipe
    self.num_images_per_prompt = num_images_per_prompt
    self.max_embeddings_multiples = max_embeddings_multiples

  def Encode(self, text_model: layered_diffusion.TextModel, text: str, default_encoding:layered_diffusion.StandardEncoding):
      # text_model, default_encodingは不使用。引数の仕様を合わせるだけのプレースホルダ
      print("using LPWEncoding")
      # 与えらえたpromptだけをencodeする
      return self.pipe._encode_prompt(
        prompt=text,
        device=self.pipe._execution_device,
        num_images_per_prompt=self.num_images_per_prompt,
        do_classifier_free_guidance=False, # promptのembeddingだけ欲しい
        negative_prompt=None,
        max_embeddings_multiples=self.max_embeddings_multiples,
      )

# Long Prompt Weighting にマスク操作を追加
class StableDiffusionLPWMaskPipeline(lpw_stable_diffusion.StableDiffusionLongPromptWeightingPipeline):
  def __init__(
      self,
      vae: AutoencoderKL,
      text_encoder: CLIPTextModel,
      tokenizer: CLIPTokenizer,
      unet: UNet2DConditionModel,
      scheduler: SchedulerMixin,
      safety_checker: StableDiffusionSafetyChecker,
      feature_extractor: CLIPFeatureExtractor,
      requires_safety_checker: bool = True,
  ):
      super().__init__(
          vae=vae,
          text_encoder=text_encoder,
          tokenizer=tokenizer,
          unet=unet,
          scheduler=scheduler,
          safety_checker=safety_checker,
          feature_extractor=feature_extractor,
          requires_safety_checker=requires_safety_checker,
      )
      
  def prepare_latents(self, image, timestep, batch_size, height, width, dtype, device, generator, latents=None):
    init_latents_orig = None
    noise = None

    if image is None:
      shape = (
          batch_size,
          self.unet.in_channels,
          height // self.vae_scale_factor,
          width // self.vae_scale_factor,
      )

      if latents is None:
        # txt2img
        #print("prepare_latents: for txt2img")
        if device.type == "mps":
            # randn does not work reproducibly on mps
            latents = torch.randn(shape, generator=generator, device="cpu", dtype=dtype).to(device)
        else:
            latents = torch.randn(shape, generator=generator, device=device, dtype=dtype)
        latents = latents * self.scheduler.init_noise_sigma
        return latents, None, None
      else:
        # img2img using latents
        #print("prepare_latents: for img2img using latents")
        if latents.shape != shape:
          raise ValueError(f"Unexpected latents shape, got {latents.shape}, expected {shape}")
        latents = latents.to(device)
        # scale the initial noise by the standard deviation required by the scheduler
        latents = latents * self.scheduler.init_noise_sigma
        init_latents_orig = latents
        init_latents = torch.cat([latents], dim=0) # make a copy

    else:
      # img2img using an image
      #print("prepare_latents: for img2img using an image")
      init_latent_dist = self.vae.encode(image).latent_dist
      init_latents = init_latent_dist.sample(generator=generator)
      init_latents = 0.18215 * init_latents
      init_latents = torch.cat([init_latents] * batch_size, dim=0)
      init_latents_orig = init_latents

    # add noise to latents using the timesteps
    shape = init_latents.shape
    if device.type == "mps":
        noise = torch.randn(shape, generator=generator, device="cpu", dtype=dtype).to(device)
    else:
        noise = torch.randn(shape, generator=generator, device=device, dtype=dtype)
    latents = self.scheduler.add_noise(init_latents, noise, timestep)

    return latents, init_latents_orig, noise

  @torch.no_grad()
  def __call__(
      self,
      prompt: Union[str, List[str]],
      negative_prompt: Optional[Union[str, List[str]]] = None,
      image: Union[torch.FloatTensor, PIL.Image.Image] = None,
      mask_image: Union[torch.FloatTensor, PIL.Image.Image] = None,
      height: int = 512,
      width: int = 512,
      num_inference_steps: int = 50,
      guidance_scale: float = 7.5,
      strength: float = 0.8,
      num_images_per_prompt: Optional[int] = 1,
      eta: float = 0.0,
      generator: Optional[torch.Generator] = None,
      latents: Optional[torch.FloatTensor] = None,
      max_embeddings_multiples: Optional[int] = 3,
      output_type: Optional[str] = "pil",
      return_dict: bool = True,
      callback: Optional[Callable[[int, int, torch.FloatTensor], None]] = None,
      is_cancelled_callback: Optional[Callable[[], bool]] = None,
      callback_steps: Optional[int] = 1,
      mask_timing: Optional[str] = "last",
      mask_output:bool = True,
      **kwargs,
  ):
      """ 追加引数:
        mask_output = True : 最終出力にマスクをかけるかどうか
        mask_timing: Optional["last", "first", "none"] = "last" : 生成ステップ内でマスクをかけるタイミング。絵柄が少し変化する
          "last": Diffusersベース。デノイズ後、ステップの最後にadd_noise()＆マスクを行い、次のステップの入力に渡す。背景が崩れやすいため mask_output=True と併用すると良い
          "first": AUTOMATIC1111 web UIに似た方法。ステップの最初にadd_noise()＆マスクし、その後デノイズする。背景が残りつつ画質が良い
          "none": ステップ中にマスクをかけない。背景が大きく書き換わるが画質は良い。mask_output=False と併用すると良い画が得られる
      """
      if (mask_timing is not None) and not (mask_timing in ["last", "first", "none"]):
          raise ValueError(f"mask_timing got {mask_timing} , expected 'last', 'first' or 'none' ")

      # 0. Default height and width to unet
      height = height or self.unet.config.sample_size * self.vae_scale_factor
      width = width or self.unet.config.sample_size * self.vae_scale_factor

      # 1. Check inputs. Raise error if not correct
      self.check_inputs(prompt, height, width, strength, callback_steps)

      # 2. Define call parameters
      batch_size = 1 if isinstance(prompt, str) else len(prompt)
      device = self._execution_device
      # here `guidance_scale` is defined analog to the guidance weight `w` of equation (2)
      # of the Imagen paper: https://arxiv.org/pdf/2205.11487.pdf . `guidance_scale = 1`
      # corresponds to doing no classifier free guidance.
      do_classifier_free_guidance = guidance_scale > 1.0

      # 3. Encode input prompt
      text_embeddings = self._encode_prompt(
          prompt,
          device,
          num_images_per_prompt,
          do_classifier_free_guidance,
          negative_prompt,
          max_embeddings_multiples,
      )
      dtype = text_embeddings.dtype

      # 4. Preprocess image and mask
      if isinstance(image, PIL.Image.Image):
          image = lpw_stable_diffusion.preprocess_image(image)
      if image is not None:
          image = image.to(device=self.device, dtype=dtype)
      if isinstance(mask_image, PIL.Image.Image):
          mask_image = lpw_stable_diffusion.preprocess_mask(mask_image, self.vae_scale_factor)
      if mask_image is not None:
          mask = mask_image.to(device=self.device, dtype=dtype)
          mask = torch.cat([mask] * batch_size * num_images_per_prompt)
      else:
          mask = None

      # 5. set timesteps
      self.scheduler.set_timesteps(num_inference_steps, device=device)
      timesteps, num_inference_steps = self.get_timesteps(num_inference_steps, strength, device, image is None)
      latent_timestep = timesteps[:1].repeat(batch_size * num_images_per_prompt)

      # 6. Prepare latent variables
      latents, init_latents_orig, noise = self.prepare_latents(
          image,
          latent_timestep,
          batch_size * num_images_per_prompt,
          height,
          width,
          dtype,
          device,
          generator,
          latents,
      )

      # 7. Prepare extra step kwargs. TODO: Logic should ideally just be moved out of the pipeline
      extra_step_kwargs = self.prepare_extra_step_kwargs(generator, eta)

      self.diffusers_style = False
      # 8. Denoising loop
      for i, t in enumerate(self.progress_bar(timesteps)):

          # AUTOMATIC1111 style, adding noise at the first
          if mask_timing == "first":
            if mask is not None:
                # masking
                init_latents_proper = self.scheduler.add_noise(init_latents_orig, noise, torch.tensor([t]))
                latents = (init_latents_proper * mask) + (latents * (1 - mask))

          # expand the latents if we are doing classifier free guidance
          latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
          latent_model_input = self.scheduler.scale_model_input(latent_model_input, t)

          # predict the noise residual
          noise_pred = self.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

          # perform guidance
          if do_classifier_free_guidance:
              noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
              noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

          # compute the previous noisy sample x_t -> x_t-1
          latents = self.scheduler.step(noise_pred, t, latents, **extra_step_kwargs).prev_sample

          # diffusers and unstable diffusion style, adding noise at the last
          if mask_timing == "last":
            # in the implementation of unstable_diffusion, skipping the adding noise at the last step
            if (mask is not None) and (i < len(timesteps)-1 ):
                # masking
                init_latents_proper = self.scheduler.add_noise(init_latents_orig, noise, torch.tensor([t]))
                latents = (init_latents_proper * mask) + (latents * (1 - mask))

          # call the callback, if provided
          if i % callback_steps == 0:
              if callback is not None:
                  callback(i, t, latents)
              if is_cancelled_callback is not None and is_cancelled_callback():
                  return None

      # 9. Post-processing

      # mask directly (both unstable diffusion and AUTOMATIC1111)
      if (mask is not None) and (mask_output == True):
          # masking
          latents = (init_latents_orig * mask) + (latents * (1 - mask))

      image = self.decode_latents(latents)

      # 10. Run safety checker
      image, has_nsfw_concept = self.run_safety_checker(image, device, text_embeddings.dtype)

      # 11. Convert to PIL
      if output_type == "pil":
          image = self.numpy_to_pil(image)

      if not return_dict:
          return image, has_nsfw_concept

      return StableDiffusionPipelineOutput(images=image, nsfw_content_detected=has_nsfw_concept)



In [ ]:
#@title 画像生成モジュール用のデータコンテナ

import copy
from distutils.util import strtobool

class GeneratedFileInfo:
  def __init__(self, prompt="", negative_prompt="",
    scheduler_name="", step_count=0, guidance_scale=0, seed=0, width=0, height=0,
    batch_size=0, batch_count=0, grid_column_count=0, enable_safety_checker=True, eta=0.0, resize=0.25, latents_type=None, extra_param = {}
  ):
    self.path = ""
    self.thumbnailBytes: bytes = None
    self.imageBytes: bytes = None
    # set parameters
    self.prompt = prompt
    self.negative_prompt = negative_prompt
    self.scheduler_name = scheduler_name
    self.step_count = step_count
    self.guidance_scale = guidance_scale
    self.seed = seed
    self.width = width
    self.height = height
    self.batch_size = batch_size
    self.batch_count = batch_count
    self.grid_column_count = grid_column_count
    self.enable_safety_checker = enable_safety_checker
    self.eta = eta
    self.resize = resize
    self.latents_type = latents_type
    self.extra_param = extra_param

  def clone(self):
    cloned = self.clone_light()
    cloned.extra_param = copy.deepcopy(self.extra_param)
    return cloned

  # バイナリデータ(imageBytes, thumbnailBytes, extra_param['image'], extra_param['mask_image'], extra_param['latents'])はコピーしないので必要に応じてコピーすること
  def clone_light(self):
    cloned = GeneratedFileInfo()
    cloned.path = copy.deepcopy(self.path)
    # set parameters
    cloned.prompt = copy.deepcopy(self.prompt)
    cloned.negative_prompt = copy.deepcopy(self.negative_prompt)
    cloned.scheduler_name = copy.deepcopy(self.scheduler_name)
    cloned.step_count = copy.deepcopy(self.step_count)
    cloned.guidance_scale = copy.deepcopy(self.guidance_scale)
    cloned.seed = copy.deepcopy(self.seed)
    cloned.width = copy.deepcopy(self.width)
    cloned.height = copy.deepcopy(self.height)
    cloned.batch_size = copy.deepcopy(self.batch_size)
    cloned.batch_count = copy.deepcopy(self.batch_count)
    cloned.grid_column_count = copy.deepcopy(self.grid_column_count)
    cloned.enable_safety_checker = copy.deepcopy(self.enable_safety_checker)
    cloned.eta = copy.deepcopy(self.eta)
    cloned.resize = copy.deepcopy(self.resize)
    cloned.latents_type = copy.deepcopy(self.latents_type)
    # extra_paramのうちバイナリデータではないものを複製
    for key,val in self.extra_param.items():
      if key in ["image", "mask_image", "latents"]:
        continue
      cloned.extra_param[key] = copy.deepcopy(val)
    
    return cloned
    
  # 埋め込み用のパラメータ文字列を生成
  def create_parameter_text(self):
  #    f"Steps: {step_count}, Sampler: {scheduler_name}, CFG scale: {guidance_scale}, Seed: {seed}, Size: {width}x{height}, Model hash: {a2a802b2}",
    param = ", ".join([
      f"Steps: {self.step_count}",
      f"Sampler: {self.scheduler_name}",
      f"CFG scale: {self.guidance_scale}",
      f"Seed: {self.seed}",
      f"Size: {self.width}x{self.height}",
    ])
    if 'strength' in self.extra_param:
      param += f", Denoising strength: {self.extra_param['strength']}"
    if self.latents_type is not None:
      param += f", Masked content: {self.latents_type}"
    if 'mask_timing' in self.extra_param:
      param += f", Mask timing: {self.extra_param['mask_timing']}"
    if 'mask_output' in self.extra_param:
      param += f", Mask output: {self.extra_param['mask_output']}"

    return "\n".join([
      self.prompt,
      f"Negative prompt: {self.negative_prompt}",
      param
    ])

  @classmethod
  def parse_parameter_text(cls, parameter_text):
    param_dict = dict()

    negative_prompt_begin = parameter_text.find("Negative prompt: ")
    # "Negative prompt:" の直前までが通常のプロンプト
    prompt = parameter_text[0:negative_prompt_begin]
    param_dict['Prompt'] = prompt
    other_str = parameter_text[negative_prompt_begin:len(parameter_text)]

    # "Steps:" の直前までがネガティブプロンプト
    step_begin = parameter_text.find("Steps:")
    negative_prompt_pair = parameter_text[negative_prompt_begin:step_begin]
    negative_prompt = negative_prompt_pair.replace('Negative prompt: ', '', 1)
    param_dict['Negative prompt'] = negative_prompt

    # "foo: **, " の形式で設定パラメータが続く
    param_dict_str = parameter_text[step_begin:len(parameter_text)]
    param_pairs = param_dict_str.split(',')
    for pair in param_pairs:
      kv = pair.split(':')
      if len(kv) == 2:
        key,value = kv
        # ,:の区切りで分割するだけだと先頭や末尾に空白が残るので、それらを除去
        # 'Euler a' のように語中に空白があるケースもあるので、replace()ではなくstrip()を使用
        param_key = key.strip()
        param_value = value.strip()
        param_dict[param_key] =param_value
      else:
        print(f"Warning: skip key-value {kv}")
      
    # 必要に応じて数値文字列を数値に変換
    conv = {
      'Prompt': lambda x: x,
      'Negative prompt': lambda x: x,
      'Steps': lambda x: int(x),
      'Sampler': lambda x: x, # 文字列をそのまま取得
      'CFG scale': lambda x: float(x),
      'Seed': lambda x: int(x),
      # Sizeをタプルに変換。 "Size: 512x512" のように記述されている
      'Size': lambda x: tuple([int(s) for s in x.split('x')]),
      'Denoising strength': lambda x: float(x),
      'Masked content': lambda x: x,
      'Mask timing': lambda x: x,
      'Mask output': lambda x: bool(strtobool(x)),
    }

    result = dict()
    for key,val in param_dict.items():
      if key in conv:
        result[key] = conv[key](val)
    
    return result

  @classmethod
  def from_parameter_text(cls, parameter_text):
    param_dict = cls.parse_parameter_text(parameter_text)
    latents_type = None
    extra_param = {}
    if 'Denoising strength' in param_dict:
      extra_param['strength'] = param_dict['Denoising strength']
    if 'Masked content' in param_dict:
      latents_type = param_dict['Masked content']
    if 'Mask timing' in param_dict:
      extra_param['mask_timing'] = param_dict['Mask timing']
    if 'Mask output' in param_dict:
      extra_param['mask_output'] = param_dict['Mask output']

    return GeneratedFileInfo(
      prompt = param_dict['Prompt'],
      negative_prompt = param_dict['Negative prompt'],
      scheduler_name = param_dict['Sampler'],
      step_count = param_dict['Steps'],
      guidance_scale = param_dict['CFG scale'],
      seed = param_dict['Seed'],
      width = param_dict['Size'][0],
      height = param_dict['Size'][1],
      #batch_size = param_dict[''],
      #batch_count = param_dict[''],
      #grid_column_count = param_dict[''],
      #enable_safety_checker = param_dict[''],
      #eta = param_dict[''],
      latents_type = latents_type,
      extra_param = extra_param,
    )

  def save_image(self, image, filepath):
    parameter_text = self.create_parameter_text()
    save_image(image, filepath, parameter_text)
    with io.BytesIO() as buf:
      save_image(image, buf, parameter_text)
      self.imageBytes = buf.getvalue()

  def save_thumbnail(self, image, resize):
    parameter_text = self.create_parameter_text()
    thumbnail = image.resize(size=(int(image.width*resize),int(image.height*resize)) )
    with io.BytesIO() as buf:
      save_image(thumbnail, buf, parameter_text)
      self.thumbnailBytes = buf.getvalue()



In [ ]:
from transformers.modeling_utils import init_empty_weights
#@title 画像生成モジュール

class BackEnd:
  def __init__(self):
    # LayeredDiffusionPipelineのConnect()による設定ではモデルのリビジョン違い等に対応できないため、手動でPipelineをセットアップ
    self.pipe: StableDiffusionPipeline = None
    self.pipeBuilder: layered_diffusion.LayeredDiffusionPipeline = None
    self.model = ""
    self.revision = ""
    self.device_to = ""
    self.attention_slicing = False
    self.safety_checker = False
    self.xformers_memory_efficient_attention = False
    # ファイル名につかう時間文字列のフォーマット
    self.timestr_format="%H%M%S_%y%b%d"
    # 更新するペイン
    self.setupPane = None
    self.txt2imgPane = None
    self.img2imgPane = None
    self.inpaintPane = None
    self.pnginfoPane = None
    
    # サンプリング手法（ノイズ除去スケジューラ）
    # https://huggingface.co/docs/diffusers/api/schedulers#implemented-schedulers
    # diffuserでは PNDM Scheduler がデフォルト
    # https://huggingface.co/blog/stable_diffusion
    # https://torch.classcat.com/2022/11/13/huggingface-blog-stable-diffusion/

    #TODO: Karrasタイムステップを用いる手法のSchedulerを実装。特にk-DPM++2M Karras
    self.schedulerDict = {
        "DPM++ 2M": {"SchedulerClass":DPMSolverMultistepScheduler, "desc":"Multistep DPM-Solver++ (DPM:diffusion probabilistic models)"},
        "Euler a": {"SchedulerClass":EulerAncestralDiscreteScheduler, "desc":"Euler Ancestral scheduler  aka k-euler-a"},
        "Euler": {"SchedulerClass":EulerDiscreteScheduler, "desc":"Euler scheduler  aka k-euler"},
        "DPM": {"SchedulerClass":DPMSolverMultistepScheduler, "desc":"Multistep DPM-Solver (DPM:diffusion probabilistic models)"},
        "DPM++ 2S": {"SchedulerClass":DPMSolverSinglestepScheduler , "desc":"Singlestep DPM-Solver++ (DPM:diffusion probabilistic models)"},
        "DDIM": {"SchedulerClass":DDIMScheduler, "desc":"Denoising diffusion implicit models"},
        "DDPM": {"SchedulerClass":DDPMScheduler, "desc":"Denoising diffusion probabilistic models"},
        "PNDM": {"SchedulerClass":PNDMScheduler, "desc":"Pseudo numerical methods for diffusion models  runwayMLモデルのデフォルト"},
        "IPNDM": {"SchedulerClass":IPNDMScheduler, "desc":"Improved Pseudo numerical methods for diffusion models"},
        "LMS": {"SchedulerClass":LMSDiscreteScheduler, "desc":"Linear multistep scheduler for discrete beta schedules  aka k-LMS"},
        "Heun 1": {"SchedulerClass":HeunDiscreteScheduler, "desc":"k-diffusion, Algorithm 1 of Karras et. al."},
        "KDPM2": {"SchedulerClass":KDPM2DiscreteScheduler, "desc":"DPM 2, log interpolation"},
        "KDPM2 a": {"SchedulerClass":KDPM2AncestralDiscreteScheduler, "desc":"KDPM2 Ancestral"},
        "KarrasVe": {"PipelineClass":KarrasVePipeline, "desc":"Variance exploding, stochastic sampling from Karras et. al"},
        "ScoreSdeVe": {"PipelineClass":ScoreSdeVePipeline, "desc":"variance exploding stochastic differential equation (VE-SDE) scheduler"},
        #"ScoreSdeVp":  {"PipelineClass":, "desc":"variance preserving stochastic differential equation (VP-SDE) scheduler"},
    }

  def setup(self, model, revision, device_to, output_dir, default_width, default_height, attention_slicing, xformers_memory_efficient_attention, safety_checker, using_layered_diffusion):
    self.model = model # "stabilityai" などのリポジトリ名
    self.revision = revision # "fp16" などのリビジョン名
    self.device_to = device_to
    self.attention_slicing = attention_slicing
    self.xformers_memory_efficient_attention = xformers_memory_efficient_attention #未対応、メンバのみ用意
    self.safety_checker = safety_checker
    self.safety_checker_obj = None
    self.using_layered_diffusion = using_layered_diffusion
    self.output_dir = output_dir

    #ディレクトリを生成しておく
    os.makedirs(self.output_dir, exist_ok=True)

    # 画像生成に用いるスケジューラ（sampler）をあらかじめ生成しておく
    #  It is deprecated to pass a pretrained model name or path to `from_config`.
    #  If you were trying to load a scheduler, please use (Scheduluerクラス名).from_pretrained(...) instead. 
    # https://note.com/npaka/n/n5fd3d8ecf1e6 を参照した
    for (k,v) in self.schedulerDict.items():
      klass = self.schedulerDict[k]
      if "SchedulerClass" in klass:
        klass["scheduler"] = klass["SchedulerClass"].from_pretrained(model, subfolder="scheduler")
    
    # construct a pipeline
    # TODO:別のパイプラインを指定できるようにする（img2img, inpaint, KarrasVeなど）

    self.pipe = StableDiffusionLPWMaskPipeline.from_pretrained(
      self.model,
      torch_dtype=torch.float16,
      revision=self.revision,
    ).to(self.device_to)

    # safety_checkerを退避しておく    
    self.safety_checker_obj = self.pipe.safety_checker
    # ロード済みのcomponentsを持っておき、別のパイプラインに切り替える時に再利用する
    self.components = self.pipe.components

    # 所要時間が増える代わりにGPUメモリ使用量を抑える
    if self.attention_slicing:
      self.pipe.enable_attention_slicing()
    # xformersによるメモリ軽減を図る
    #if self.xformers_memory_efficient_attention:
      #self.pipe.enable_xformers_memory_efficient_attention()

    # LayeredDiffusionPipelineのConnect()による設定ではモデルのリビジョン違い等に対応できないため、手動でセットアップ
    self.pipeBuilder = layered_diffusion.LayeredDiffusionPipeline()
    #TODO: publicメソッドがあればそちらを使用する
    self.pipeBuilder._dataset = self.model
    self.pipeBuilder._SetPipeline(self.pipe)

    #テキストエンコーダをセット
    #self.textEncoder = layered_diffusion.ShiftEncoding(rotate=False, reverse=False)
    self.textEncoder = LPWEncoding(self.pipe)

    #txt2imgのサンプラーリストとwidth,height初期値を更新
    self.txt2imgPane.set_param( list(self.schedulerDict.keys()), default_width, default_height )
    self.img2imgPane.set_param( list(self.schedulerDict.keys()), default_width, default_height )
    self.inpaintPane.set_param( list(self.schedulerDict.keys()), default_width, default_height )


  def word_count(self, prompt):
    if self.pipe is None:
      return 0
    untruncated_ids = self.pipe.tokenizer(prompt, padding="longest", return_tensors="pt").input_ids
    return untruncated_ids.shape[-1]

  def max_word_count(self):
    if self.pipe is None:
      return 0
    multiple = 1
    if hasattr(self.textEncoder, "max_embeddings_multiples"):
      multiple = self.textEncoder.max_embeddings_multiples
    return multiple * (self.pipe.tokenizer.model_max_length-2)+2 #先頭と末尾の2wordを引いてから倍数をかける

  # Masked Content の "latent nothing"を実装（未完成）
  def create_latent_nothing(self, info, generator=None):
    # https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py から抜粋・変更
    value=0.0
    device = self.pipe.device
    dtype = self.pipe.text_encoder.dtype
    num_channels_latents = self.pipe.unet.in_channels
    batch_size = info.batch_size * 1 # num_images_per_prompt = 1 as default
    shape = (batch_size, num_channels_latents, info.height // self.pipe.vae_scale_factor, info.width // self.pipe.vae_scale_factor)
    return torch.full(shape, value, device=device, dtype=dtype).to(self.device_to)

  # Masked Content の "latent noise"を実装
  def create_latent_noise(self, info, generator):
    # https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py から抜粋・変更
    device = self.pipe.device
    dtype = self.pipe.text_encoder.dtype
    num_channels_latents = self.pipe.unet.in_channels
    batch_size = info.batch_size * 1 # num_images_per_prompt = 1 as default
    shape = (batch_size, num_channels_latents, info.height // self.pipe.vae_scale_factor, info.width // self.pipe.vae_scale_factor)
    return torch.randn(shape, device=device, dtype=dtype, generator=generator).to(self.device_to)


  """
  inpaint, img2img, txt2imgの画像生成で使う初期データを決める
  Stable Diffusionにおいてはいずれの機能も手順は同じで、初期データが違う。
  本実装では、img2img, txt2imgはinpaint(InpaintLegacy)の特殊化としている。
  img2img, txt2imgの方が処理が分かりやすいため先に記述。
  """
  def img2img(self, info):
    image = info.extra_param["image"]
    strength = info.extra_param["strength"]
    new_info = info.clone_light()

    new_info.extra_param["image"] = image
    new_info.extra_param["mask_image"] = None
    new_info.extra_param["latents"] = None

    # 画像全体を初期データとする
    init_latent = layered_diffusion.ByImage(image=image, strength=1.0)
    # layer: 初期データに対するデノイズ指定のレイヤー。プロンプトやCFG scaleの指定はこちらで行う
    # プロンプトのレイヤは1枚だけ
    layer = layered_diffusion.Layer(
      prompt=info.prompt,
      negative_prompt=info.negative_prompt,
      cfg_scale=info.guidance_scale,
      strength=strength,
    )
    return init_latent, layer, new_info

  def txt2img(self, info):
    # latent noiseを初期データとする
    # txt2imgではRandomlyを使ってもGUI側でのlatent noiseとほぼ同一の画像となる
    #（inpaintでRandomlyを使うとAUTOMATIC1111版に近い画像が出ない。inpaint()での実装を参照）

    new_info = info.clone_light()
    new_info.extra_param["image"] = None
    new_info.extra_param["mask_image"] = None
    new_info.extra_param["latents"] = None

    # init_latent = layered_diffusion.Randomly(symmetric=False, strength=1.0)
    init_latent = None # Layered Diffusionでの修正を反映。initなしの場合はRandomlyが使用される
    # プロンプトのレイヤは1枚だけ
    layer = layered_diffusion.Layer(
      prompt=info.prompt,
      negative_prompt=info.negative_prompt,
      cfg_scale=info.guidance_scale,
      strength=1.0,
    )
    return init_latent, layer, new_info

  # inpaintのMasked Contentで "latent noise", "latent nothing" を初期値とする場合に
  # 潜在空間上でマスクをかけた画像データを用意
  # stable_diffusion_pipeline.py より引用・改変
  def prepare_image_latents(self, info, g_cpu, noise_func, strength):
    device = self.pipe.device
    dtype = self.pipe.text_encoder.dtype

    # 4. Preprocess image and mask
    image = lpw_stable_diffusion.preprocess_image(info.extra_param["image"])
    image = image.to(device=device, dtype=dtype)
    mask_image = lpw_stable_diffusion.preprocess_mask(info.extra_param["mask_image"], self.pipe.vae_scale_factor)
    mask_image = mask_image.to(device=device, dtype=dtype)

    # 画像にVAEを適用してlatentsを生成（このとき乱数発生）
    init_latent_dist = self.pipe.vae.encode(image).latent_dist
    init_latents = init_latent_dist.sample(generator=g_cpu)
    init_latents = 0.18215 * init_latents
    init_latents = torch.cat([init_latents] * info.batch_size, dim=0)

    # latent noiseを生成
    noise_fill = noise_func(info, g_cpu)

    # マスクをかける。マスク適用範囲のみ背景とノイズをアルファブレンド
    rate = (1.0-mask_image) * strength
    latents = ((1.0-rate) * init_latents) + (rate * noise_fill)
    return latents

  # inpaintでの設定。マスク適用の設定により処理を分岐
  def inpaint(self, info):

    # 初期ノイズの種類を設定。
    # ノイズの種類に対してマスクを適用したものが初期latents
    """
    fill: ピクセルのgray( RGB=(128,128,128) )
    original: imageの変換後のノイズをそのまま使う。InpaintPipelineLegacyでimageを与えたときのデフォルト値
    latent noise: 潜在空間上でのランダムノイズ。GUI側でノイズ生成
    latent nothing: 潜在空間上でのzeros
    mode channel: 独自実装。マスク範囲の画素の最頻値を使う。チャネルごとに最頻値を取った値をRGB値とする
    top image: 独自実装。指定した画像をマスクで切り抜いて上(top)に上書き
    randomly: layered diffusionのみ。 Layered Diffusion側で latent noise を生成
    """
    image = info.extra_param["image"]
    mask = info.extra_param["mask_image"]
    strength = info.extra_param["strength"]
    latents_type = info.latents_type
    new_info = info.clone_light()
    new_info.extra_param["image"] = image
    new_info.extra_param["mask_image"] = mask
    new_info.extra_param["latents"] = None
    #print(new_info.latents_type, new_info.extra_param)

    # init_latent: 初期データの指定。[Initializer] も可能で、
    # init_latent[0] に対し、init_latent[1]から順に複数のInitializerを、それぞれ指定したマスクをかけて重ねることが可能

    # init_latentのデフォルト値。imageの変換後のノイズをそのまま使う（originalに相当）
    # strength (Denoise strength): 拡散の順過程でノイズをかける回数の割合[0.0-1.0]。
    # 回数 = numstep * strength、回数が多いほど初期状態がよりノイズに近づき、元の画像とはより異なる画像が生成される。
    latent_image = layered_diffusion.ByImage(image=image, strength=strength)
    init_latent = latent_image
    randgen = self.pipeBuilder.scheduler.generator()

    # latentsの種別を取得
    if latents_type == "original":
      pass
    elif latents_type == "fill":
      pass
    elif latents_type == "mode channel":
      pass
    elif latents_type == "latent noise":
      #マスクして上書き
      if self.using_layered_diffusion:
        # AUTOMATIC1111版からの再現性のため、1. 画像にVAEを適用してlatentsを生成（このとき乱数発生）、2. latent noiseを生成 の順序で処理を行う。
        #latent_noise = layered_diffusion.Randomly(symmetric=False, mask_by=mask, strength=strength)
        latent_noise = layered_diffusion.ByLatents(latents=self.create_latent_noise(info,randgen), mask_by=mask, strength=strength)
        init_latent = [latent_image, latent_noise]
      else:
        latents = self.prepare_image_latents(info, randgen, self.create_latent_noise, strength)
        new_info.extra_param["latents"] = latents
        new_info.extra_param["image"] = None # set image None to use latents on generating

    elif latents_type == "latent nothing":
      #マスクして上書き
      if self.using_layered_diffusion:
        latent_nothing = layered_diffusion.ByLatents(latents=self.create_latent_nothing(info), mask_by=mask, strength=strength)
        init_latent = [latent_image, latent_nothing]
      else:
        # 画像からlatentsを生成（乱数発生）
        latents = self.prepare_image_latents(info, randgen, self.create_latent_nothing, strength)
        new_info.extra_param["latents"] = latents
        new_info.extra_param["image"] = None # set image None to use latents on generating

    elif latents_type == "randomly":
      # Layered Diffusion側で latent noise を生成。AUTOMATIC1111版とは得られる画像が異なる
      # symmetric: 対称的なノイズにするかどうか。Trueでだいたい対称的になる
      # TODO: symmetricのチェックボックスの対応
      random_pixel = layered_diffusion.Randomly(symmetric=False, mask_by=mask, strength=strength)
      init_latent = [latent_image, random_pixel]

    else:
      #未指定＝デフォルト値
      pass

    # layer: 初期データに対するデノイズ指定のレイヤー。プロンプトやCFG scaleの指定はこちらで行う
    layer = layered_diffusion.Layer(
      prompt=info.prompt,
      negative_prompt=info.negative_prompt,
      cfg_scale=info.guidance_scale,
      strength=1.0, #TODO
      mask_by = mask,
    )
    #print(new_info.latents_type, new_info.extra_param)
    return init_latent, layer, new_info


  # 生成した画像ファイル情報を返す
  # info.seed == -1 （シード値を自動決定）を与えた時は、自動決定されたシード値をinfo.seedにも設定する
  # seed_index: seedに加算、batch_count内で何回目か
  def generate_image_progressive(self, info, pipeMethod):
    # パイプラインを切り替える
    # TODO? : "KarrasVe"などPipelineとSchedulerが不可分な場合の対応
    if "PipelineClass" in self.schedulerDict[info.scheduler_name]:
      pipelineClass = self.schedulerDict[info.scheduler_name]["PipelineClass"]
      if issubclass(pipelineClass, StableDiffusionPipeline) and pipelineClass != type(self.pipe):
        self.pipe = pipelineClass(self.components)
        self.pipeBuilder.pipe = self.pipe
    else:
      # 切り替えない
      self.pipe.scheduler = self.schedulerDict[info.scheduler_name]["scheduler"]

    # layered_diffusion側のschedulerを設定
    sche = layered_diffusion.Scheduler()
    sche.scheduler = self.pipe.scheduler
    sche.have_max_timesteps = False
    sche._rand_seed = None
    sche._generator = None
    self.pipeBuilder.scheduler = sche

    #シードをセット
    if(info.seed < 0):
      #シードを得る
      self.pipeBuilder._ResetGenerator(None)
      info.seed = self.pipeBuilder.GetRandSeed()
    self.pipeBuilder._ResetGenerator(info.seed)

    # 設定をパイプラインに反映
    prompt_array = [info.prompt] * info.batch_size
    negative_prompt_array = [info.negative_prompt] * info.batch_size

    if self.safety_checker == True and self.pipe.safety_checker == None:
      self.pipe.safety_checker = self.safety_checker_obj
    else:
      self.pipe.safety_checker = None

    # 所要時間が増える代わりにGPUメモリ使用量を抑える
    if self.attention_slicing:
      self.pipe.enable_attention_slicing()
    else:
      self.pipe.disable_attention_slicing()

    if self.using_layered_diffusion:
      # 機能に対応した初期データとレイヤ構造を得る
      init_latent, layer, _ = pipeMethod(info)

      # initialize:初期データの指定。initialize=[Initializer] も可能で、initialize[0] に対し、initialize[1]から順に複数のinitを、
      # それぞれ指定したマスクをかけて重ねることが可能。（InitializerList.Merge()が使用される）
      # rmm_dump: デバッグ用のレイヤごと出力
      outImage, rmm_dump = self.pipeBuilder(
          num_steps = info.step_count,
          size = (info.width, info.height),
          default_encoding = self.textEncoder,
          rand_seed = info.seed,
          eta = info.eta,
          initialize = init_latent, #上記注
          iterate = layer,
      ) #TODO: 返ってくるのは1バッチ分だけ？
    
    else:
      # infoを書き換える
      _, _, new_info = pipeMethod(info)
      # 画像生成
      outImage = self.pipe.__call__(
        prompt=[info.prompt] * info.batch_size,
        negative_prompt=[info.negative_prompt] * info.batch_size,
        width=info.width,
        height=info.height,
        num_inference_steps=info.step_count,
        guidance_scale=info.guidance_scale,
        eta=info.eta,
        generator=self.pipeBuilder.scheduler.generator(),
        max_embeddings_multiples=3,
        callback = None,
        is_cancelled_callback = None,
        **new_info.extra_param # image, mask_image, latents, strength, (inpaintのみ mask_timing, mask_output)
      ).images[0]
    
    # 生成データをファイルに保存    
    timestr=time.strftime(self.timestr_format)
    if info.batch_size > 1:
      # 1バッチずつファイルに保存
      filepath = os.path.join(self.output_dir, f"batch_{timestr}_seed{info.seed}.png")
      grid_image = create_batch_image(outImage, info.width, info.height, info.grid_column_count)
      info.save_image(grid_image, filepath)
      info.save_thumbnail(grid_image, info.resize)
    else:
      # 1回分のファイルを保存
      filepath = os.path.join(self.output_dir, f"{timestr}_seed{info.seed}.png")
      info.save_image(outImage, filepath)
      info.save_thumbnail(outImage, info.resize)
      """
      # デバッグ出力を保存
      rmm_filepath = os.path.join(self.output_dir, f"{timestr}_rmm_seed{info.seed}.png")
      save_image(rmm_dump, rmm_filepath, "")
      """
    
    return info

  def synchronize(self):
    torch.cuda.synchronize(device=self.pipe.device)

  def generate_grid(self, info_list, grid_info):
    timestr=time.strftime(self.timestr_format)
    filepath = os.path.join(self.output_dir, f"grid_{timestr}_seed{grid_info.seed}.png")
    single_batch_images = [Image.open(io.BytesIO(info.imageBytes)) for info in info_list]
    grid_image = create_batch_image(single_batch_images, grid_info.width, grid_info.height, grid_info.grid_column_count)
    grid_info.save_image(grid_image, filepath)
    grid_info.save_thumbnail(grid_image, grid_info.resize)

    return grid_image


# 処理モジュールのインスタンスを生成
sharedBackEnd = BackEnd()


In [ ]:
#@title 共通GUIコンポーネント

# 処理する画像のアップローダ
class ImageUploader:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self, parent, width, height, mode):
    self.parent = parent
    #画像データ、PIL Imageのインスタンス
    self.src_image = None
    self.mode = mode
    #ウィジェット
    self.src_image_widget = widgets.Image(width=f"{width}", height=f"{height}")
    self.src_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="source img")
    self.src_upload_button.observe(self.on_src_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動  
    self.src_upload_button.layout=Layout(width=f"{width}")

  #ファイルアップロード完了時に元画像を更新する
  def on_src_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.src_image = Image.open(io.BytesIO(data)).convert(self.mode)
    self.src_image_widget.value = data
  
  def set_layout(self):
    self.layout = widgets.VBox([self.src_upload_button, self.src_image_widget])
    return self.layout

class GeneratePane:
  def __init__(self, parent):
    self.sampler_name_list = []
    self.is_image_generating = False # 画像生成中フラグ
    self.generated_file_list = []
    self.parent = parent
    self.prompt_textarea = widgets.Textarea()
    self.prompt_textarea.observe(self.on_prompt_update, names='value') #valueを監視してイベントハンドラを起動
    self.wordcount_label = widgets.Label(value=f"prompt 0/0")

    self.negative_prompt_textarea = widgets.Textarea()
    self.negative_prompt_textarea.observe(self.on_negative_prompt_update, names='value') #valueを監視してイベントハンドラを起動
    self.negative_wordcount_label = widgets.Label(value=f"negative 0/0")

    self.generate_button = widgets.Button(description="Generate")
    self.generate_button.on_click(self.parent.generate)

    self.interrupt_button = widgets.Button(description="Interrupt", disabled=True, layout=Layout(width="50%") )
    self.stop_button = widgets.Button(description="Stop", disabled=True, layout=Layout(width="50%") )

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    self.prompt_textarea.value = info.prompt
    self.negative_prompt_textarea.value = info.negative_prompt

  def on_prompt_update(self, change):
    word_count = sharedBackEnd.word_count(change.new)
    max_word_count = sharedBackEnd.max_word_count()
    self.wordcount_label.value = f"prompt {word_count}/{max_word_count}"

  def on_negative_prompt_update(self, change):
    word_count = sharedBackEnd.word_count(change.new)
    max_word_count = sharedBackEnd.max_word_count()
    self.negative_wordcount_label.value = f"negative {word_count}/{max_word_count}"

  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    #入力要素のレイアウトを設定
    self.prompt_textarea.placeholder="prompt"
    self.prompt_textarea.layout = Layout(width="100%",height="7em")
    self.negative_prompt_textarea.placeholder="Negative prompt"
    self.negative_prompt_textarea.layout = Layout(width="100%",height="7em", border="solid 0px")
    self.generate_button.style={"button_color":"lightskyblue"}
    #generate_stack = widgets.Stack([generate_button, interrupt_button], selected_index=0, layout = Layout(width="100%",height="100%"))
    self.generate_stack = widgets.VBox([\
      self.generate_button, widgets.HBox([self.interrupt_button, self.stop_button], layout = Layout(width="100%",height="50%")) \
    ])

    self.layout = widgets.HBox(layout=Layout(border="solid 0px"), children=[
      # 生成ボタン類
      widgets.VBox(layout=Layout(width="15%",height="auto", border="solid 0px"), children=[
        self.generate_stack,
        self.wordcount_label,
        self.negative_wordcount_label,
      ]),
      # プロンプト・ネガティブプロンプト
      widgets.VBox(layout=Layout(width="100%",height="100%", border="solid 0px"), children=[
        widgets.Box(layout = Layout(width="100%",height="100%"), children=[
          self.prompt_textarea,
        ]),
        widgets.Box(layout = Layout(width="100%",height="100%"), children=[
          self.negative_prompt_textarea,
        ]),
      ]),
    ])
    return self.layout


class SettingPane:
  def __init__(self):
    #生成パラメータ
    self.sampling_step_slider = widgets.IntSlider(min=0, max=150, value=20)
    self.sampling_method = widgets.Dropdown()
    self.CFG_scale_slider = widgets.FloatSlider(min=0, max=25.0, value=7.0)
    self.seed_input = widgets.IntText(value=-1)
    #保存画像のサイズ・枚数
    self.width_slider = widgets.IntSlider(min=8, max=2048, value=512, step=8)
    self.height_slider = widgets.IntSlider(min=8, max=2048, value=512, step=8)
    self.batch_size_input = widgets.BoundedIntText(value=1, min=1) # 1回の作業の生成枚数。枚数分だけGPUメモリを使用
    self.batch_count_input = widgets.BoundedIntText(value=6, min=1) # 生成作業を何回行うか。回数分だけ処理時間はかかるが、GPUは1回分の使用量で済む
    self.grid_column_count_input = widgets.BoundedIntText(value=3, min=1) # 画像gridの列の数

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    self.sampling_step_slider.value = info.step_count
    self.width_slider.value = info.width
    self.height_slider.value = info.height
    self.CFG_scale_slider.value = info.guidance_scale
    self.seed_input.value =  info.seed
    # リストにない場合は反映しない
    if info.scheduler_name in sharedBackEnd.schedulerDict.keys() and info.scheduler_name in self.sampling_method.options :
      self.sampling_method.value = info.scheduler_name

  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.sampler_name_list = sampler_name_list
    self.sampling_method.options=sampler_name_list
    self.sampling_method.value=sampler_name_list[0]
    self.width_slider.value = default_width
    self.height_slider.value = default_height

  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    self.sampling_step_slider.style={"description_width":"10em"}
    self.sampling_step_slider.layout=Layout(width="90%", border="solid 0px")
    self.sampling_step_slider.description="steps"
    self.sampling_method.style={"description_width":"10em"}
    self.sampling_method.layout=Layout(width="90%", border="solid 0px")
    self.sampling_method.description="method"

    self.CFG_scale_slider.style={"description_width":"10em"}
    self.CFG_scale_slider.layout=Layout(width="90%", border="solid 0px")
    self.CFG_scale_slider.description="CFG scale"
    self.seed_input.style={"description_width":"10em"}
    self.seed_input.layout=Layout(width="90%", border="solid 0px")
    self.seed_input.description="seed"

    self.width_slider.style={"description_width":"10em"}
    self.width_slider.layout=Layout(width="90%", border="solid 0px")
    self.width_slider.description="width"
    self.height_slider.style={"description_width":"10em"}
    self.height_slider.layout=Layout(width="90%", border="solid 0px")
    self.height_slider.description="height"

    self.batch_size_input.style={"description_width":"10em"}
    self.batch_size_input.layout=Layout(width="90%", border="solid 0px")
    self.batch_size_input.description="batch Size"
    self.batch_count_input.style={"description_width":"10em"}
    self.batch_count_input.layout=Layout(width="90%", border="solid 0px")
    self.batch_count_input.description="batch Count"
    self.grid_column_count_input.style={"description_width":"10em"}
    self.grid_column_count_input.layout=Layout(width="90%", border="solid 0px")
    self.grid_column_count_input.description="grid columns"

    # txt2imgの設定ペイン
    self.setting_pane_content = {
      "sampling":widgets.VBox([self.sampling_step_slider, self.sampling_method]),
      "quality params":widgets.VBox([self.CFG_scale_slider,self.seed_input]),
      "image size":widgets.VBox([
        self.width_slider, self.height_slider,
        self.batch_size_input, self.batch_count_input,
        self.grid_column_count_input
      ]),
      }
    self.setting_pane = widgets.VBox(layout = Layout(width="100%", height="100%"))
    # tabを1個ずつ追加（tab切り替えは不使用。tabの部分をタイトル表示にのみ使用）
    children_array = list(self.setting_pane_content.values())
    self.setting_pane.children = [widgets.Tab([wd]) for wd in children_array]
    for i,title in enumerate(self.setting_pane_content.keys()):
      self.setting_pane.children[i].set_title(0, title)

    self.layout = widgets.Box([self.setting_pane], layout=Layout(width="100%", border="solid 0px") )
    return self.layout

class OutputPane:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.thumbnail_size = 64 # px
    self.thumbnail_button_width = 16 # px
    self.thumbnail_button_height = 64 # px
    # 出力画像ビューワの部分
    self.focus_image = widgets.Image(width="512", height="512")
    self.focus_image_box = widgets.Box()
    self.thumbnail_viewer = widgets.VBox()
    self.generated_file_list = []
    self.output = widgets.Output()

    #GUIデバッグ用
    self.viewer_debug = False
    #self.viewer_debug = True
    if self.viewer_debug:
      self.viewer_debug = True
      #string_generated_file_list = ["thumbnail01.png", "thumbnail02.png",]
      string_generated_file_list = ["test_image.png","test_image.png"]
      #string_generated_file_list = []
      for filename in string_generated_file_list:
        # 1回分のファイルを保存
        image = Image.open(filename)
        resize=0.25
        thumbnail = image.resize(size=(int(image.width*resize),int(image.height*resize)) )
        info = GeneratedFileInfo(
            prompt="", negative_prompt="", scheduler_name="",
            step_count=0, guidance_scale=0, seed=0,
            width=image.width, height=image.height, batch_size=0, batch_count=0, grid_column_count=0
        )
        with io.BytesIO() as buf:
          save_image(image, buf, "")
          info.imageBytes = buf.getvalue()
        with io.BytesIO() as buf:
          save_image(thumbnail, buf, "")
          info.thumbnailBytes = buf.getvalue()
        self.generated_file_list.append(info)
  
  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    self.focus_image_box.children = [self.focus_image]
    self.focus_image_box.layout = Layout(width="520", height="520") #64x64 px thumbnail
    self.focus_image.layout.object_fit = 'contain'
    self.focus_image.layout.object_position = 'top'
    self.thumbnail_viewer.layout = Layout(width="100", height="520") #64x64 px thumbnail

    self.layout = widgets.VBox(layout=Layout(border="lightgray solid 1px"), children=[
          widgets.Label(value="right click to download an image or thumbnail / click a 🖼️ button to show a full image", layout=Layout(height="2em")), # notion
          widgets.HBox([self.focus_image, self.thumbnail_viewer], layout=Layout(width="100%", height="520", border="solid 0px") ), #生成画像
          widgets.Box([self.output],layout=Layout(width="100%", height="16em", border="gray solid 1px")), # output
        ])
    return self.layout

  #"generate"ボタン押下時に画像生成し、1枚ずつサムネイル表示
  def generate_progressive(self, ui_info, pipeMethod):
    # UIと絶縁するため、ディープコピーを取る
    info = ui_info.clone_light()
    #extra_paramはシャローコピー
    info.extra_param = ui_info.extra_param

    self.output.clear_output()
    with self.output:
      #TODO: 生成中はボタンを押せないようにする
      #TODO: implement GUI of batch_size, batch_count and grid_column_count
      if self.viewer_debug:
        count = len(self.generated_file_list)
      else:
        self.generated_file_list = []
        count = info.batch_count
    
      # 先にサムネイルリストを生成
      #batch_size == 1のときはgridも生成
      if info.batch_size == 1:
        thumbnail_count = count+1
      else:
        thumbnail_count = count
      self.create_thumbnail_list(thumbnail_count)

      # 1枚ずつ生成し、その都度サムネイルを表示
      init_seed = info.seed
      count_seed = info.seed
      for i in range(count):
        # シード値が-1（自動決定）でかつ初回の生成では、画像生成側でシード値を決める
        if not(init_seed < 0 and i == 0):
          # 2回目以降またはシード値固定の場合は、すでにあるシード値に加算する
          count_seed = init_seed + i

        # 反復ごとに絶縁するため、ディープコピーを取る
        new_info = info.clone_light()
        #extra_paramはシャローコピー
        new_info.extra_param = info.extra_param

        new_info.seed = count_seed
        if self.viewer_debug:
          file = self.generated_file_list[i]
        else:
          # 画像生成
          sharedBackEnd.generate_image_progressive(new_info, pipeMethod)
          self.generated_file_list.append(new_info)
          # 画像生成時に決定したシード値を得る
          if init_seed < 0 and i == 0:
            count_seed = new_info.seed
            init_seed = new_info.seed #gridに書き込むためこれも書き換えておく
        
        if new_info.batch_size == 1:
          index = i+1 #children[0]はgrid
        else:
          index = i
        #サムネイルを表示（画像データをセット）
        self.set_thumbnail(index, new_info.thumbnailBytes)
      
    #self.output.clear_output()
    with self.output:
      #gridを作成
      if info.batch_size == 1:
        grid_info = info.clone_light()
        grid_info.extra_param = info.extra_param
        grid_info.seed = init_seed
        grid = sharedBackEnd.generate_grid(self.generated_file_list, grid_info)
        #先頭に追加
        self.generated_file_list.insert(0, grid_info)
        #サムネイルを表示（画像データをセット）
        self.set_thumbnail(0, grid_info.thumbnailBytes)
    
    self.is_image_generating = False
    return self.generated_file_list

  #先にサムネイルリストを生成
  def create_thumbnail_list(self, count):
      thumbnail_widget_list = []
      self.thumbnail_button_list = [] #クリックイベントハンドラ内でボタンに対応するインデックスを得るためのリスト
      #空のサムネイルリストを生成
      for i in range(count):
        wd = self.create_viewer_progressive()
        thumbnail_widget_list.append(wd)
      #先にサムネイルリストをビューワに追加
      self.thumbnail_viewer.children = thumbnail_widget_list

  #サムネイルに画像データをセット
  def set_thumbnail(self, index, thumbnailBytes):
    self.thumbnail_viewer.children[index].children[0].value = thumbnailBytes

  #サムネイルボタンのクリックイベントハンドラを生成
  def focus_handler(self, change):
    button = change
    i = self.thumbnail_button_list.index(button)
    self.focus_image.value = self.generated_file_list[i].imageBytes


  # 空のサムネイルを追加
  def create_viewer_progressive(self):
    image = widgets.Image(width=self.thumbnail_size, height=self.thumbnail_size)
    image.layout.object_fit = 'contain'

    # クリックしてfocus_imageを更新するボタンを生成
    button = widgets.Button(description="🖼️")
    button.layout = Layout(width=f"{self.thumbnail_button_width}", height=f"{self.thumbnail_button_height}")
    button.on_click(self.focus_handler)
    self.thumbnail_button_list.append(button)

    # 画像とボタンを並べたものを1セットとしてサムネイルに追加
    return widgets.HBox([image, button])


In [ ]:
#@title setupタブ

# モデル使用時のコンポーネント類は https://huggingface.co/fooID/barModel/
# 下の model_index.json をデフォルト設定として参照して内部的にロードされる。ほかのモデルでも同様の仕組み

class SetupUIView:
  def __init__(self):
    self.model_revision = {
      # stable diffusion official
      "stabilityai/stable-diffusion-2-1-base":{"default":"fp16", "revision":["main","bf16","fp16"]},
      "stabilityai/stable-diffusion-2-1":{"default":"fp16", "revision":["main","bf16","fp16"], "width":768, "height":768},
      #"stabilityai/stable-diffusion-2-depth":{"default":"fp16", "revision":["main","fp16"]},
      "runwayml/stable-diffusion-v1-5":{"default":"fp16", "revision":["main","bf16","flax","fp16","onnx"]},
      "naclbit/trinart_stable_diffusion_v2":{"default":"diffusers-115k", "revision":["main","diffusers-115k","diffusers-95k","diffusers-60k"]},
      "naclbit/trinart_characters_19.2m_stable_diffusion_v1":{"default":"main", "revision":["main"]},
      #"":{"default":"main", "revision":["main"]},
    }
    self.device_to_list=["cuda", "cpu"]
    self.width=512
    self.height=512

    self.model_select = widgets.Dropdown(options=list(self.model_revision.keys()), value=list(self.model_revision.keys())[0] )
    self.revision_select = widgets.Dropdown(options=[])
    self.enable_other_model_check = widgets.Checkbox(value=False)
    self.other_model_input = widgets.Text(value="")
    self.other_revision_input = widgets.Text(value="main")
    self.enable_layered_diffusion_check = widgets.Checkbox(value=True)

    #on_clickを持たないため、valueを監視してイベントハンドラを起動
    def on_using_layered_diffusion_check(change):
      sharedBackEnd.using_layered_diffusion = change.new
    self.enable_layered_diffusion_check.observe(on_using_layered_diffusion_check, names='value')

    #modelに応じてrevisionリストを変える
    def update_revision(change):
      model = change['new']
      self.revision_select.options = self.model_revision[model]["revision"]
      self.revision_select.value = self.model_revision[model]["default"]
    self.model_select.observe(update_revision, 'value')
    #revision_selectの初期値を設定
    update_revision({'new':self.model_select.value})

    self.device_to_select = widgets.Dropdown(options=self.device_to_list, value=self.device_to_list[0])
    self.enable_attention_slicing_check = widgets.Checkbox(value=False)
    self.xformers_memory_efficient_attention_check = widgets.Checkbox(value=False)
    self.enable_safety_checker_check = widgets.Checkbox(value=True)

    #on_clickを持たないため、valueを監視してイベントハンドラを起動
    def on_safety_checker_check(change):
      sharedBackEnd.safety_checker = change.new    
    self.enable_safety_checker_check.observe(on_safety_checker_check, names='value')

    def on_enable_attention_slicing_check(change):
      sharedBackEnd.attention_slicing = change.new
    self.enable_attention_slicing_check.observe(on_enable_attention_slicing_check, names='value')

    def on_xformers_memory_efficient_attention_check(change):
      sharedBackEnd.xformers_memory_efficient_attention = change.new    
    self.xformers_memory_efficient_attention_check.observe(on_xformers_memory_efficient_attention_check, names='value')

    self.setup_button = widgets.Button(description="Setup")
    self.setup_button.on_click(self.on_setup)
    self.output = widgets.Output()

  def set_layout(self):
    self.model_select.style={"description_width":"10em"}
    self.model_select.layout=Layout(width="36em")
    self.model_select.description="preset model ID"
    self.revision_select.description="revision"
    self.revision_select.layout=Layout(width="15em")
    self.enable_other_model_check.style={"description_width":"6em"}
    self.enable_other_model_check.layout=Layout(width="14em")
    self.enable_other_model_check.description="other model"
    self.other_model_input.style={"description_width":"0em"}
    self.other_model_input.layout=Layout(width="36em")
    self.other_model_input.description=""
    self.other_revision_input.style={"description_width":"4.5em"}
    self.other_revision_input.layout=Layout(width="12em")
    self.other_revision_input.description="revision"
    self.enable_layered_diffusion_check.description="use layered_diffusion"

    self.device_to_select.style={"description_width":"10em"}
    self.device_to_select.layout=Layout(width="16em")
    self.device_to_select.description="device to"
    self.enable_attention_slicing_check.style={"description_width":"10em"}
    self.enable_attention_slicing_check.layout=Layout(width="22em")
    self.enable_attention_slicing_check.description="attention slicing"
    self.xformers_memory_efficient_attention_check.style={"description_width":"10em"}
    self.xformers_memory_efficient_attention_check.layout=Layout(width="12em")
    self.xformers_memory_efficient_attention_check.description="xformers memory efficient attention"
    self.enable_safety_checker_check.style={"description_width":"10em"}
    self.enable_safety_checker_check.layout=Layout(width="22em")
    self.enable_safety_checker_check.description="safety checker"
    self.setup_button.style={"button_color":"lightgreen"}

    # setupの設定ペイン
    self.setting_pane_content = {
      "model": widgets.VBox(layout=Layout(border="solid 0px"), children=[\
        widgets.HBox(layout=Layout(border="solid 0px"), children=[ \
          self.model_select,
          self.revision_select,
        ]),
        widgets.HBox(layout=Layout(border="solid 0px"), children=[ \
          self.enable_other_model_check,
          self.other_model_input,
          self.other_revision_input,
        ]),
        widgets.HBox([self.enable_layered_diffusion_check]),
      ]),
      "device":widgets.VBox([
        self.device_to_select,
        widgets.HBox([
          self.enable_attention_slicing_check,
          #self.xformers_memory_efficient_attention_check,
          self.enable_safety_checker_check,
        ]),
      ]),
    }
    self.setting_pane = widgets.VBox(layout = Layout(width="62em", height="100%"))
    # tabを1個ずつ追加（tab切り替えは不使用。tabの部分をタイトル表示にのみ使用）
    children_array = list(self.setting_pane_content.values())
    self.setting_pane.children = [widgets.Tab([wd]) for wd in children_array]
    for i,title in enumerate(self.setting_pane_content.keys()):
      self.setting_pane.children[i].set_title(0, title)

    self.layout = widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.setting_pane,
      self.setup_button,
      widgets.Box([self.output],layout=Layout(width="100%", height="20em", border="gray solid 1px"))
    ])

    return self.layout

  def on_setup(self, change):
    # model, revisionをチェック
    if self.enable_other_model_check.value == True:
      model = self.other_model_input.value
      revision = self.other_revision_input.value
    else:
      model = self.model_select.value
      revision = self.revision_select.value
    
    # デフォルトwidth, heightを変更
    if "width" in self.model_revision[self.model_select.value]:
      self.width = self.model_revision[self.model_select.value]["width"]
    if "height" in self.model_revision[self.model_select.value]:
      self.height = self.model_revision[self.model_select.value]["height"]
    # 画像生成モジュールをセットアップ
    self.output.clear_output()
    with self.output:
      sharedBackEnd.setup(
        model=model, revision=revision, device_to=self.device_to_select.value, 
        output_dir="/content/output", default_width=self.width, default_height=self.height,
        attention_slicing=self.enable_attention_slicing_check.value,
        xformers_memory_efficient_attention=self.xformers_memory_efficient_attention_check.value,
        safety_checker=self.enable_safety_checker_check.value,
        using_layered_diffusion=self.enable_layered_diffusion_check.value,
      )

# ペインのインスタンスを生成
setupPane = SetupUIView()



In [ ]:
#@title txt2imgタブ

class Txt2imgPaneView:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.generatePane = GeneratePane(self)
    self.settingPane = SettingPane()
    self.outputPane = OutputPane()

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info):
    self.generatePane.reflectPNGInfo(info)
    self.settingPane.reflectPNGInfo(info)

  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.settingPane.set_param(sampler_name_list, default_width, default_height)

  def generate(self, change):
    info = GeneratedFileInfo(
      prompt = self.generatePane.prompt_textarea.value,
      negative_prompt = self.generatePane.negative_prompt_textarea.value,
      scheduler_name = self.settingPane.sampling_method.value,
      step_count = self.settingPane.sampling_step_slider.value,
      guidance_scale = self.settingPane.CFG_scale_slider.value,
      seed = self.settingPane.seed_input.value,
      width = self.settingPane.width_slider.value,
      height = self.settingPane.height_slider.value,
      batch_size = self.settingPane.batch_size_input.value,
      batch_count = self.settingPane.batch_count_input.value,
      grid_column_count = self.settingPane.grid_column_count_input.value,
      eta=0.0,
      resize=0.25
    )
    self.outputPane.generate_progressive(info, sharedBackEnd.txt2img)



  # 入力要素の定義とレイアウト設定を分離する
  # 全体のレイアウトを設定
  def set_layout(self):
    self.layout = \
    widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.generatePane.set_layout(),
      # 設定／生成画像
      widgets.HBox(layout=Layout(border="solid 0px"), children=[
        widgets.Box([self.settingPane.set_layout()], layout=Layout(width="45%")),
        widgets.Box([self.outputPane.set_layout()], layout=Layout(width="55%")),
      ]),
    ])
    return self.layout


# txt2imgペインのインスタンスを生成
txt2imgPane = Txt2imgPaneView()


In [ ]:
#@title img2imgタブ

class Img2imgPane:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self, parent):
    self.parent = parent
    #画像データ
    self.src_image = None
    #ウィジェット
    self.src_image_widget = widgets.Image(width="120", height="120")
    self.denoise_strength_slider = widgets.FloatSlider(description="Denoising strength", min=0, max=1.0, value=0.75)
    self.src_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="source img")
    self.src_upload_button.observe(self.on_src_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    if 'strength' in info.extra_param:
      self.denoise_strength_slider.value = info.extra_param['strength']

  # txt2imgペインのレイアウトを設定
  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    content = widgets.VBox(layout=Layout(width="100%"), children=[
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox([self.src_upload_button, self.src_image_widget]),
      ]),
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        self.denoise_strength_slider,        
      ])
    ])

    self.denoise_strength_slider.style={"description_width":"10em"}
    self.denoise_strength_slider.layout=Layout(width="20em", border="solid 0px")
    self.src_upload_button.layout=Layout(width="100")

    tab = widgets.Tab([content])
    tab.set_title(0, "img2img")

    # 全体のレイアウトを設定
    self.layout = \
    widgets.Box(layout=Layout(width="100%", border="solid 0px"), children=[
      tab,
    ])

    return self.layout

  #ファイルアップロード完了時に元画像を更新する
  def on_src_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.src_image = Image.open(io.BytesIO(data)).convert("RGB")
    self.src_image_widget.value = data
    
class Img2imgPaneView:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.generatePane = GeneratePane(self)
    self.img2imgPane = Img2imgPane(self)
    self.settingPane = SettingPane()
    self.outputPane = OutputPane()

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info):
    self.generatePane.reflectPNGInfo(info)
    self.settingPane.reflectPNGInfo(info)
    self.img2imgPane.reflectPNGInfo(info)


  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.settingPane.set_param(sampler_name_list, default_width, default_height)

  def generate(self, change):
    info = GeneratedFileInfo(
      prompt = self.generatePane.prompt_textarea.value,
      negative_prompt = self.generatePane.negative_prompt_textarea.value,
      scheduler_name = self.settingPane.sampling_method.value,
      step_count = self.settingPane.sampling_step_slider.value,
      guidance_scale = self.settingPane.CFG_scale_slider.value,
      seed = self.settingPane.seed_input.value,
      width = self.settingPane.width_slider.value,
      height = self.settingPane.height_slider.value,
      batch_size = self.settingPane.batch_size_input.value,
      batch_count = self.settingPane.batch_count_input.value,
      grid_column_count = self.settingPane.grid_column_count_input.value,
      eta=0.0,
      resize=0.25,
      extra_param = {
        "strength": self.img2imgPane.denoise_strength_slider.value,
        "image": self.img2imgPane.src_image,
      }
    )
    # 画像を1枚ずつ生成
    self.outputPane.generate_progressive(info, sharedBackEnd.img2img)



  # 入力要素の定義とレイアウト設定を分離する
  # 全体のレイアウトを設定
  def set_layout(self):
    self.layout = \
    widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.generatePane.set_layout(),
      # 設定／生成画像
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox(layout=Layout(width="45%", border="solid 0px"), children=[
          self.img2imgPane.set_layout(),
          self.settingPane.set_layout(),
        ]),
        widgets.Box([self.outputPane.set_layout()], layout=Layout(width="50%", border="solid 0px"))
      ]),
    ])
    return self.layout



#Inpaintペインのインスタンスを生成
img2imgPane = Img2imgPaneView()


In [ ]:
#@title Inpaintタブ


class InpaintPane:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self, parent):
    self.parent = parent
    self.model_sample_size = 512
    #画像データ
    self.src_image = None
    self.mask_image = None
    self.top_image = None
    self.inpaint_image = None
    #ウィジェット
    self.src_image_widget = widgets.Image(width="100", height="100")
    self.mask_image_widget = widgets.Image(width="100", height="100", layout=Layout(border="dotted 1px skyblue") )
    self.top_image_widget = widgets.Image(width="100", height="100")
    self.overlay_widget = widgets.Image(width="120", height="120")
    self.mask_switch = widgets.RadioButtons(description="Inpaint areas", value="white", options=["black", "white"])
    self.mask_switch.observe(self.on_mask_switched, names='value') #valueを監視してイベントハンドラを起動
    self.masked_content_select = widgets.Dropdown(description="Masked content", options=["fill", "original", "mode channel", "latent noise", "randomly", "top image"])
    self.masked_content_select.observe(self.on_MaskedContent_upload, names='value') #valueを監視してイベントハンドラを起動
    self.denoise_strength_slider = widgets.FloatSlider(description="Denoising strength", min=0, max=1.0, value=0.75)
    self.src_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="back img")
    self.src_upload_button.observe(self.on_src_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動
    self.mask_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="mask")
    self.mask_upload_button.observe(self.on_mask_upload, names='value') #valueを監視してイベントハンドラを起動
    self.top_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="top img")
    self.top_upload_button.observe(self.on_top_upload, names='value') #valueを監視してイベントハンドラを起動
    self.mask_timing_select = widgets.Dropdown(description="Mask timing", options=["last", "first", "none"])
    self.enable_mask_output_check = widgets.Checkbox(value=True, description="Mask output")

  def set_param(self, using_layered_diffusion):
    self.using_layered_diffusion = using_layered_diffusion
    #TODO: settingペイン側で"use layered_diffusion"を切り替えた時にdisableを反映させる（notifyの仕組みが必要）
    if using_layered_diffusion == True:
      pass
      #self.mask_timing_select.disable = True
      #self.enable_mask_output_check.disable = True
    else:
      # LPW pipelineを使う場合は、randomlyは使用不可
      self.masked_content_select.options=["fill", "original", "mode channel", "latent noise", "latent nothing", "top image"]
      #self.mask_timing_select.disable = False
      #self.enable_mask_output_check.disable = False

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info:GeneratedFileInfo):
    if (info.latents_type is not None) and (info.latents_type in self.masked_content_select.options):
      self.masked_content_select.value = info.latents_type

    if 'strength' in info.extra_param:
      self.denoise_strength_slider.value = info.extra_param['strength']
    if 'mask_timing' in info.extra_param:
      self.mask_timing_select.value = info.extra_param['mask_timing']
    if 'mask_output' in info.extra_param:
      self.enable_mask_output_check.value = info.extra_param['mask_output']

  # txt2imgペインのレイアウトを設定
  # 入力要素の定義とレイアウト設定を分離する
  def set_layout(self):
    inpaint_content = widgets.VBox(layout=Layout(width="100%"), children=[
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox([self.src_upload_button, self.src_image_widget]),
        widgets.VBox([self.mask_upload_button, self.mask_image_widget]),
        widgets.VBox([self.top_upload_button, self.top_image_widget]),
      ]),
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        self.overlay_widget,
        widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
          widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
            self.mask_switch,
            self.masked_content_select,
            self.denoise_strength_slider,
          ]),
          widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
            widgets.Label(value="LPW Pipeline setting:"),
            self.mask_timing_select,
            self.enable_mask_output_check,
          ]),
        ])
      ])
    ])

    self.mask_switch.layout.display="flex"
    self.mask_switch.style={"description_width":"10em"}
    self.masked_content_select.style={"description_width":"10em"}
    self.masked_content_select.layout=Layout(width="20em", border="solid 0px")
    self.denoise_strength_slider.style={"description_width":"10em"}
    self.denoise_strength_slider.layout=Layout(width="95%", border="solid 0px")
    self.src_upload_button.layout=Layout(width="100")
    self.mask_upload_button.layout=Layout(width="100")
    self.top_upload_button.layout=Layout(width="100")
    self.mask_timing_select.layout=Layout(width="12em", border="solid 0px")
    #self.enable_mask_output_check.style={"description_width":"6em"}

    inpaint_tab = widgets.Tab([inpaint_content])
    inpaint_tab.set_title(0, "inpaint")

    # 全体のレイアウトを設定
    self.layout = \
    widgets.Box(layout=Layout(width="100%", border="solid 0px"), children=[
      inpaint_tab,
    ])   

    return self.layout

  #ファイルアップロード完了時にマスクを更新する
  def on_mask_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.mask_image_widget.value = data
    self.on_mask_switched(None)
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時にマスクを更新する
  def on_mask_switched(self, change):
    if self.mask_image_widget.value == None:
      return #何もしない
    
    mask = Image.open(io.BytesIO(self.mask_image_widget.value)).convert("RGB")
    if(self.mask_switch.value == "black"):
      #マスクを反転する
      self.mask_image = ImageChops.invert(mask)
    else:
      self.mask_image = mask
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時に元画像を更新する
  def on_src_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.src_image = Image.open(io.BytesIO(data)).convert("RGB")
    self.src_image_widget.value = data
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時に元画像を更新する
  def on_top_upload(self, change):
    data = fileUploader_load_png(new_value=change.new) #bytes
    self.top_image = Image.open(io.BytesIO(data)).convert("RGB")
    self.top_image_widget.value = data
    self.overlay_mask(self.masked_content_select.value)

  #ファイルアップロード完了時に元画像を更新する
  def on_MaskedContent_upload(self, change):
    self.overlay_mask(change.new)    

  # 元画像にマスクを重ねて表示
  def overlay_mask(self, latents_type):
    if self.src_image == None or self.mask_image == None:
      return #何もしない

    #マスクを元画像に重ねて表示
    self.inpaint_image = self.mask_fill_image(latents_type)

    #表示用の画像データをバッファに出力
    with io.BytesIO() as buf:
      if not (latents_type == "fill" or latents_type == "original" or latents_type == "mode channel" or latents_type == "top image"):
        # マスクを白抜きで表示
        image = self.src_image.copy()
        mask = self.mask_image.convert("L")
        image.paste(self.mask_image, mask=mask)
        image.save(buf, format="png")
      else:
        self.inpaint_image.save(buf, format="png")
      # 表示用の画像データをウィジェットに設定
      self.overlay_widget.value = buf.getvalue()

  # 元画像をマスクで切り抜いて塗りつぶす
  def mask_fill_image(self, latents_type):
    image = self.src_image.copy()
    mask = self.mask_image.convert("L")
    fill = None
    
    if latents_type == "fill":
      fill = Image.new(mode="RGB", size=(self.src_image.width, self.src_image.height), color="gray")
    if latents_type == "top image" and self.top_image != None:
      fill = self.top_image
    elif latents_type == "mode channel":
      # imageのマスク範囲におけるチャネルごとの最頻値を使う
      histo = self.src_image.histogram(mask=mask)
      ch_mode_index = [ max(rng, key=histo.__getitem__) for rng in [range(0,256), range(256,512), range(512,768)] ] #インデックスが画素値に相当
      color = (ch_mode_index[0], ch_mode_index[1]-256, ch_mode_index[2]-512) #インデックスのオフセットを引く
      #print("ch_mode_index", ch_mode_index, "color", color)
      fill = Image.new(mode="RGB", size=(self.src_image.width, self.src_image.height), color=color)
    elif latents_type == "original":
      # 何もしない
      pass
    else:
      # 何もしない
      pass

    # マスクをかける画像がなければ、元の画像（背景）を返す
    if fill == None:
      return self.src_image
    else:
      # imageにマスクをかけて貼り付け
      # TODO: Mask Blurを実装
      image.paste(fill, mask=mask)
      return image

    
class InpaintPaneView:
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self):
    self.generatePane = GeneratePane(self)
    self.inpaintPane = InpaintPane(self)
    self.settingPane = SettingPane()
    self.outputPane = OutputPane()

  # PNGInfoから得たパラメータ情報を反映させる
  def reflectPNGInfo(self, info):
    self.generatePane.reflectPNGInfo(info)
    self.settingPane.reflectPNGInfo(info)
    self.inpaintPane.reflectPNGInfo(info)

  # setupペインでのセットアップ内容を反映させる
  def set_param(self, sampler_name_list, default_width, default_height):
    self.settingPane.set_param(sampler_name_list, default_width, default_height)
    self.inpaintPane.set_param(sharedBackEnd.using_layered_diffusion)

  def generate(self, change):
    info = GeneratedFileInfo(
      prompt = self.generatePane.prompt_textarea.value,
      negative_prompt = self.generatePane.negative_prompt_textarea.value,
      scheduler_name = self.settingPane.sampling_method.value,
      step_count = self.settingPane.sampling_step_slider.value,
      guidance_scale = self.settingPane.CFG_scale_slider.value,
      seed = self.settingPane.seed_input.value,
      width = self.settingPane.width_slider.value,
      height = self.settingPane.height_slider.value,
      batch_size = self.settingPane.batch_size_input.value,
      batch_count = self.settingPane.batch_count_input.value,
      grid_column_count = self.settingPane.grid_column_count_input.value,
      eta=0.0,
      resize=0.25,
      latents_type = self.inpaintPane.masked_content_select.value,
      extra_param = {
        "strength": self.inpaintPane.denoise_strength_slider.value,
        "image": self.inpaintPane.inpaint_image,
        "mask_image": self.inpaintPane.mask_image,
        "mask_timing": self.inpaintPane.mask_timing_select.value,
        "mask_output": self.inpaintPane.enable_mask_output_check.value,
      }
    )
    # 画像を1枚ずつ生成
    self.outputPane.generate_progressive(info, sharedBackEnd.inpaint)



  # 入力要素の定義とレイアウト設定を分離する
  # 全体のレイアウトを設定
  def set_layout(self):
    self.layout = \
    widgets.VBox(layout=Layout(width="100%", border="solid 0px"), children=[
      self.generatePane.set_layout(),
      # 設定／生成画像
      widgets.HBox(layout=Layout(width="100%", border="solid 0px"), children=[
        widgets.VBox(layout=Layout(width="45%", border="solid 0px"), children=[
          self.inpaintPane.set_layout(),
          self.settingPane.set_layout(),
        ]),
        widgets.Box([self.outputPane.set_layout()], layout=Layout(width="50%", border="solid 0px"))
      ]),
    ])
    return self.layout



#Inpaintペインのインスタンスを生成
inpaintPane = InpaintPaneView()



In [ ]:
#@title PNGInfoタブ

# 実際のレイアウトは別途行う
class PnginfoPaneView:
  def __init__(self, txt2imgPane, img2imgPane, inpaintPane):
    self.txt2imgPane = txt2imgPane
    self.img2imgPane = img2imgPane
    self.inpaintPane = inpaintPane
    self.param_dict = {}
    self.info = None #GeneratedFileInfo

    self.pnginfo_prompt_textarea = widgets.Textarea(placeholder="prompt", disabled=True, layout = Layout(width="100%",height="15em"))
    
    self.pnginfo_upload_button = widgets.FileUpload(accept=".png; .jpg", multiple=False, description="upload an image")
    self.pnginfo_upload_button.observe(self.on_pnginfo_upload, names='value') #FileUploadはon_clickを持たないため、valueを監視してイベントハンドラを起動
    
    self.pnginfo_upload_image = widgets.Image(width="512", height="512")
    
    self.send_to_txt2img_button = widgets.Button(description="send to txt2img",layout = Layout(width="auto") )
    self.send_to_txt2img_button.on_click(self.on_send_to_txt2img)

    self.send_to_img2img_button = widgets.Button(description="send to img2img",layout = Layout(width="auto") )
    self.send_to_img2img_button.on_click(self.on_send_to_img2img)

    self.send_to_inpaint_button = widgets.Button(description="send to inpaint",layout = Layout(width="auto") )
    self.send_to_inpaint_button.on_click(self.on_send_to_inpaint)

    self.thumbnail_image = widgets.Image(width="64", height="64")

    self.enable_edit_check = widgets.Checkbox(value=False, description="enable edit")
    self.enable_edit_check.observe(self.on_enable_edit_check, names="value")

    self.edit_save_button = widgets.Button(description="save image",layout = Layout(width="auto") )
    self.edit_save_button.on_click(self.on_edit_save)
    self.edit_save_thumbnail_button = widgets.Button(description="save thumbnail",layout = Layout(width="auto") )
    self.edit_save_thumbnail_button.on_click(self.on_edit_save_thumbnail)
    
  def set_layout(self):
    # PNG Infoのペイン
    self.layout = \
    widgets.HBox(layout=Layout(width="100%", height="100%"), children=[
      # 画像のアップロード
      widgets.VBox(layout=Layout(width="540", height="100%"), children=[
        self.pnginfo_upload_button,
         widgets.HBox([self.pnginfo_upload_image], layout=Layout(width="520", height="520")),
      ] ),
      # 画像の設定パラメータ表示、SendToボタン
      widgets.VBox(layout=Layout(width="60%", height="100%"), children=[
        widgets.HBox([self.pnginfo_prompt_textarea], layout=Layout(width="100%", height="100%")),
        widgets.HBox( [
          self.send_to_txt2img_button,
          self.send_to_img2img_button,
          self.send_to_inpaint_button,
          widgets.VBox([
            self.enable_edit_check,
            widgets.HBox( [self.edit_save_button, self.edit_save_thumbnail_button,]),
          ]),
        ]),
        widgets.HBox([
          self.thumbnail_image,
        ]),
      ]),
    ])
    return self.layout

  #ファイルアップロード完了時に画像とパラメータ表示を更新する
  def on_pnginfo_upload(self, change):
    (data, parameter_text) = update_pnginfo(new_value=change.new)
    image = Image.open(io.BytesIO(data))
    self.info = GeneratedFileInfo.from_parameter_text(parameter_text)
    self.info.imageBytes = data
    self.pnginfo_upload_image.value = data
    self.pnginfo_upload_image.width = min(512, image.width)
    self.pnginfo_upload_image.height = min(512, image.height)
    self.pnginfo_prompt_textarea.value = parameter_text
    
    # サムネイルを作成、parameter_textをそのまま保存
    resize = 0.25
    thumbnail = create_thumbnail(image=image, resize=resize)
    with io.BytesIO() as buf:
      save_image(thumbnail, buf, parameter_text)
      self.thumbnail_image.width = min(512, image.width*resize)
      self.thumbnail_image.height = min(512, image.height*resize)
      self.thumbnail_image.value = buf.getvalue()
    
    # ロード直後は常にedit無効
    self.enable_edit_check.value = False

  """
  "send to **"ボタン押下時にそれぞれのペインへ反映させる
  """
  def on_send_to_txt2img(self, remove):
    # TODO: should be event-driven
    new_info = GeneratedFileInfo.from_parameter_text(self.pnginfo_prompt_textarea.value)
    new_info.imageBytes = self.pnginfo_upload_image.value
    self.info = new_info
    txt2imgPane.reflectPNGInfo(self.info)

  def on_send_to_img2img(self, remove):
    # TODO: should be event-driven
    new_info = GeneratedFileInfo.from_parameter_text(self.pnginfo_prompt_textarea.value)
    new_info.imageBytes = self.pnginfo_upload_image.value
    self.info = new_info
    img2imgPane.reflectPNGInfo(self.info)

  def on_send_to_inpaint(self, remove):
    # TODO: should be event-driven
    new_info = GeneratedFileInfo.from_parameter_text(self.pnginfo_prompt_textarea.value)
    new_info.imageBytes = self.pnginfo_upload_image.value
    self.info = new_info
    inpaintPane.reflectPNGInfo(self.info)

  # check時に編集の有効を切り替える
  def on_enable_edit_check(self, change):
    self.pnginfo_prompt_textarea.disabled = not change.new

  def on_edit_save(self, remove):
    img = Image.open(io.BytesIO(self.pnginfo_upload_image.value))
    with io.BytesIO() as buf:
      save_image(img, buf, self.pnginfo_prompt_textarea.value)
      self.pnginfo_upload_image.value = buf.getvalue()

  def on_edit_save_thumbnail(self, remove):
    img = Image.open(io.BytesIO(self.thumbnail_image.value))
    with io.BytesIO() as buf:
      save_image(img, buf, self.pnginfo_prompt_textarea.value)
      self.thumbnail_image.value = buf.getvalue()

#PNGInfoペインのインスタンスを生成
pnginfoPane = PnginfoPaneView(txt2imgPane, img2imgPane, inpaintPane)


In [ ]:
#@title Extrasタブ内の共通GUI

from google.colab import files
import subprocess

class ExtrasSubTab:
  def __init__(self, parent, root):
    self.parent = parent
    self.rootDir = root
    self.inputDir = os.path.join(root, 'Input')
    self.outputDir = os.path.join(root, 'Output')
    self.weightsDir = os.path.join(root, 'weights')
    # 入力ディレクトリを作成
    if os.path.exists(self.inputDir) == False:
      os.mkdir(self.inputDir)
    # 出力先ディレクトリを作成
    if os.path.exists(self.outputDir) == False:
      os.mkdir(self.outputDir)

    self.generate_button = widgets.Button()
    self.generate_button.on_click(self.on_generate)
    self.outputWindow = widgets.Output()
    self.settingPane = None # 具体的な設定のGUI

    self.upload_button = widgets.Button()
    self.upload_button.on_click(self.on_upload)
    self.output_file_select = widgets.SelectionSlider(options=["keep", "remove"], value="remove")
    self.input_file_select = widgets.SelectionSlider(options=["keep", "remove"], value="remove")

    self.remove_input_file_check = widgets.Checkbox(value=True) # 生成後に入力ファイルを消す
    self.remove_output_file_check = widgets.Checkbox(value=True) # 生成前に出力ファイルを消す
    self.download_button = widgets.Button()
    self.download_button.on_click(self.on_download)

  # 設定項目のGUIを返すよう派生クラスで実装する
  def set_setting_layout(self):
    pass

  # 生成処理を派生クラスで実装する
  def generate(self):
    pass
  
  def set_layout(self):
    self.generate_button.layout=Layout(width="50%")
    self.generate_button.description="generate"
    self.generate_button.style={"button_color":"lightskyblue"}
    self.upload_button.description="upload images"
    self.download_button.description="download zip"

    self.output_file_select.description="Output files before generating"
    self.output_file_select.style={"description_width": "16em"}
    self.output_file_select.layout=Layout(width="23em")

    self.input_file_select.description="Input files after generating"
    self.input_file_select.style={"description_width": "16em"}
    self.input_file_select.layout=Layout(width="23em")

    setting_tab = widgets.Tab(layout = Layout(width="auto",height="auto"), children=[self.set_setting_layout()])
    setting_tab.set_title(0, "param")

    self.layout = \
    widgets.VBox(layout=Layout(width="100%", height="100%"), children=[
      widgets.HBox(children=[
        #self.src_uploader.set_layout(),
        widgets.Box(layout=Layout(width="50%", height="auto"), children=[
          widgets.VBox([
            widgets.HBox([self.upload_button, widgets.Label(value="(popup to the right window)")]),
            widgets.HBox([self.output_file_select, self.input_file_select]),
            setting_tab,
            widgets.HBox([self.generate_button], layout=Layout(width="auto", height="auto", margin="1em 0") ),
            self.download_button,
          ]),
        ]),
        widgets.Box([self.outputWindow],layout=Layout(width="50%", height="20em", border="gray solid 1px")), # output
      ]),
    ])
    return self.layout

  def on_generate(self, remove):
    remove_output = (self.output_file_select.value == "remove")
    remove_input = (self.input_file_select.value == "remove")

    self.outputWindow.clear_output()
    with self.outputWindow:
      # 生成前に出力ファイルを消す
      if remove_output == True:
        subprocess.run(f"rm {self.outputDir}/*", shell=True)

      self.generate()

      # 生成後に入力ファイルを消す
      if remove_input == True:
        subprocess.run(f"rm {self.inputDir}/*", shell=True)

  def on_upload(self, remove):
    with self.outputWindow:
      # colabの機能を使ってアップロードされたファイルを受け取る
      uploaded = files.upload()
      for filename in uploaded.keys():
        # ファイルをInputディレクトリに移動
        subprocess.run(f"mv {filename} {self.inputDir}", shell=True)

  # Outputディレクトリの中身を1個のzipファイルにまとめ、ダウンロード。
  # Colabでは一度に1ファイルしかダウンロードできない。
  # 複数ファイルを1個のzipファイルにまとめることで、実質的に複数ファイルを一度でダウンロードできる
  def on_download(self, remove):
    with self.outputWindow:
      # Outputディレクトリの中身を1個のzipファイルにまとめる
      zipfile = f"{self.rootDir}/Output.zip"
      subprocess.run(f"rm {zipfile}", shell=True)
      subprocess.run(f"zip -r -j {zipfile} {self.outputDir}", shell=True)
      # colabの機能を使ってダウンロード
      files.download(zipfile)


In [ ]:
#@title Extrasタブ／高解像度化処理

# HighResタブ（高解像度化）
class HighResRealESR(ExtrasSubTab):
  def __init__(self, parent, root):
    super().__init__(parent, root)
    self.model_select = widgets.Dropdown(description="model", options=[ "RealESRGAN_x4plus", "RealESRNet_x4plus", "RealESRGAN_x4plus_anime_6B", "RealESRGAN_x2plus", "realesr-animevideov3", "realesr-general-x4v3"], value="RealESRGAN_x4plus" )
    self.scale_slider = widgets.FloatSlider(min=0, max=4.0, value=4.0)
    self.face_enhance_check = widgets.Checkbox(value=False)
  
  def set_setting_layout(self):
    self.scale_slider.description="scale"
    self.face_enhance_check.description="face enhance"
    return widgets.VBox([
      self.model_select,
      self.scale_slider,
      self.face_enhance_check,
    ])

  # 高解像度化画像を生成
  # https://github.com/xinntao/Real-ESRGAN/inference_realesrgan.py からコピーして一部変更
  def generate(self):
    #!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload -o results --outscale {self.scale_slider.value} {--face_enhance}

    # パラメータ
    model_path = None
    outscale = self.scale_slider.value
    face_enhance = self.face_enhance_check.value

    #TODO: support setting of these parameters
    dni_weight = None
    denoise_strength = 1
    tile=0
    tile_pad=10
    pre_pad=0
    half=torch.cuda.is_available() #True #fp16
    gpu_id=None
    suffix = 'out' #出力ファイル名の末尾につける文字列。 foobar_{suffix}.{ext} のようなファイル名になる
    ext = 'auto' # 出力ファイルの拡張子

    # determine models according to model names
    model_name = self.model_select.value
    if model_name == 'RealESRGAN_x4plus':  # x4 RRDBNet model
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
        netscale = 4
        file_url = ['https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth']
    elif model_name == 'RealESRNet_x4plus':  # x4 RRDBNet model
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
        netscale = 4
        file_url = ['https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/RealESRNet_x4plus.pth']
    elif model_name == 'RealESRGAN_x4plus_anime_6B':  # x4 RRDBNet model with 6 blocks
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
        netscale = 4
        file_url = ['https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth']
    elif model_name == 'RealESRGAN_x2plus':  # x2 RRDBNet model
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
        netscale = 2
        file_url = ['https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth']
    elif model_name == 'realesr-animevideov3':  # x4 VGG-style model (XS size)
        model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=16, upscale=4, act_type='prelu')
        netscale = 4
        file_url = ['https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth']
    elif model_name == 'realesr-general-x4v3':  # x4 VGG-style model (S size)
        model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
        netscale = 4
        file_url = [
            'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-wdn-x4v3.pth',
            'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth'
        ]
    else:
      # do nothing
      print(f"not registered model: '{model_name}'")
      return

    # determine model paths
    model_path = os.path.join(self.weightsDir, model_name + '.pth')
    if not os.path.isfile(model_path):
      for url in file_url:
        # model_path will be updated
        model_path = load_file_from_url( url=url, model_dir=self.weightsDir, progress=True, file_name=None )

    # use dni to control the denoise strength
    if model_name == 'realesr-general-x4v3' and denoise_strength != 1:
      wdn_model_path = model_path.replace('realesr-general-x4v3', 'realesr-general-wdn-x4v3')
      model_path = [model_path, wdn_model_path]
      dni_weight = [denoise_strength, 1 - denoise_strength]

    # restorer
    # outscale は処理時に指定する
    upsampler = RealESRGANer(
      scale=netscale,
      model_path=model_path,
      dni_weight=dni_weight,
      model=model,
      tile=tile,
      tile_pad=tile_pad,
      pre_pad=pre_pad,
      half=half,
      gpu_id=gpu_id)

    if face_enhance == True:  # Use GFPGAN for face enhancement
      from gfpgan import GFPGANer
      face_enhancer = GFPGANer(
        model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
        upscale=outscale,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=upsampler)


    paths = sorted( glob.glob(os.path.join(self.inputDir, '*')) )
    for idx, path in enumerate(paths):
      imgname, extension = os.path.splitext(os.path.basename(path))
      print('Testing', idx, imgname)

      img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
      if len(img.shape) == 3 and img.shape[2] == 4:
        img_mode = 'RGBA'
      else:
        img_mode = None

      try:
        if face_enhance:
          _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
        else:
          # outscaleを指定
          output, _ = upsampler.enhance(img, outscale=outscale)
      except RuntimeError as error:
        print('Error', error)
        print('If you encounter CUDA out of memory, try to set --tile with a smaller number.')
      else:
        if ext == 'auto':
          extension = extension[1:]
        else:
          extension = ext
        if img_mode == 'RGBA':  # RGBA images should be saved in png format
          extension = 'png'
        if suffix == '':
          save_path = os.path.join(self.outputDir, f'{imgname}.{extension}')
        else:
          save_path = os.path.join(self.outputDir, f'{imgname}_{suffix}.{extension}')
        cv2.imwrite(save_path, output)

    print('complete.')


In [ ]:
#@title Extrasタブ／深度推定処理

# DepthMapタブ（深度推定）
class DepthMapMiDaS(ExtrasSubTab):
  # 入力要素・出力要素の定義を行う。実際のレイアウトはset_layout()で設定
  def __init__(self, parent, root):
    super().__init__(parent, root)

    # 公式のサンプル結果より、dpt_swin_large_384 をモデルに使用（椅子が2個とも深度推定できている、速度も悪くない）
    # 他に dpt_next_vit_large_384 が候補
    self.model_select = widgets.Dropdown(description="model", options=[
        "dpt_beit_large_512", "dpt_beit_large_384", "dpt_beit_base_384", "dpt_swin2_large_384",
        "dpt_swin2_base_384", "dpt_swin2_tiny_256", "dpt_swin_large_384", "dpt_next_vit_large_384",
        "dpt_levit_224", "dpt_large_384", "dpt_hybrid_384", "midas_v21_384", "midas_v21_small_256",
        "openvino_midas_v21_small_256" ],
        #value = "dpt_beit_large_512", #run.pyのデフォルト
        value = "dpt_swin_large_384",
    )

  def set_setting_layout(self):
    return widgets.VBox([
      self.model_select,
    ])

  # 深度画像を生成
  # https://github.com/isl-org/MiDaS/blob/master/run.py からコピーして一部変更
  def generate(self):
    #!python run.py --model_type <model_type> -model_weights <path/to/model.pt> --input_path input --output_path output

    #パラメータ
    model_type = self.model_select.value
    side = False #default value is True

    #TODO: support setting of these parameters
    optimize = None #fp32
    height = None
    square = True
    grayscale = True

    model_path = self.download_model(model_type, self.weightsDir)
    if model_path == None:
      print(f"not registered model type:{model_type}")
      return

    print("Initialize")

    # select device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device: %s" % device)

    model, transform, net_w, net_h = midas.model_loader.load_model(device, model_path, model_type, optimize, height, square)

    # get input
    if self.inputDir is not None:
      image_names = glob.glob(os.path.join(self.inputDir, "*"))
      num_images = len(image_names)
    else:
      print("No input path specified.")
      return

    print("Start processing")    

    for index, image_name in enumerate(image_names):

      print("  Processing {} ({}/{})".format(image_name, index + 1, num_images))

      # input
      original_image_rgb = utils.read_image(image_name)  # in [0, 1]
      image = transform({"image": original_image_rgb})["image"]

      # compute
      with torch.no_grad():
        prediction = process(device, model, model_type, image, (net_w, net_h), original_image_rgb.shape[1::-1],
                              optimize, False)

      # output
      if self.outputDir is not None:
        filename = os.path.join(
          self.outputDir, os.path.splitext(os.path.basename(image_name))[0] + '-' + model_type
        )
        if not side:
          utils.write_depth(filename, prediction, grayscale, bits=2)
        else:
          original_image_bgr = np.flip(original_image_rgb, 2)
          content = create_side_by_side(original_image_bgr*255, prediction, grayscale)
          cv2.imwrite(filename + ".png", content)
        utils.write_pfm(filename + ".pfm", prediction.astype(np.float32))

    print("Finished")


  #モデルをダウンロードしてパスを得る
  def download_model(self, model_type, weights_dir):
    model_path_dict = {
      "dpt_beit_large_512": ["v3_1", []],
      "dpt_beit_large_384": ["v3_1", []],
      "dpt_beit_base_384": ["v3_1", []],
      "dpt_swin2_large_384": ["v3_1", []],
      "dpt_swin2_base_384": ["v3_1", []],
      "dpt_swin2_tiny_256": ["v3_1", []],
      "dpt_swin_large_384": ["v3_1", []],
      "dpt_next_vit_large_384": ["v3_1", []],
      "dpt_levit_224": ["v3_1", []],
      "dpt_large_384": ["v3", []],
      "dpt_hybrid_384": ["v3", []],
      "midas_v21_384": ["v2_1", []],
      "midas_v21_small_256": ["v2_1", []],
      "openvino_midas_v21_small_256": ["v3_1", ["openvino_midas_v21_small_256.xml", "openvino_midas_v21_small_256.bin"] ]
      }
    model_path = ""
    filename = ""
    url_root = "https://github.com/isl-org/MiDaS/releases/download"

    if model_type in model_path_dict.keys():
      if len(model_path_dict[model_type][1]) == 0:
        filename = f"{model_type}.pt"
        filelist = [filename]
        model_path = os.path.join(weights_dir, filename)
      else:
        # for openvino
        filelist = model_path_dict[model_type][1]
        model_path = os.path.join(weights_dir, filelist[0] ) #.xml file
    else:
      return None
    
    if not os.path.isfile(model_path):
      for filename in filelist:
        url = '/'.join( [url_root, model_path_dict[model_type][0], filename] )   # https://github.com/isl-org/MiDaS/releases/download/v3_1/dpt_beit_large_512.pt
        local = os.path.join(weights_dir, filename)  # /content/MiDaS/weights/dpt_beit_large_512.pt
        print(url, local)
        torch.hub.download_url_to_file( url=url, dst=local, progress=True )
    
    return model_path



In [ ]:
#@title Extras全体のタブ
class ExtraTabView:
  def __init__(self):
    self.highResTab = HighResRealESR(self, HighRes_root)
    self.depthMapTab = DepthMapMiDaS(self, DepthMap_root)

  def set_layout(self):
    #Extra全体のサブメニューを収めるタブを生成
    top_tab_dict = {"HighRes":self.highResTab.set_layout(), "DepthMap":self.depthMapTab.set_layout() }
    top_tab = widgets.Tab(layout = Layout(width="100%",height="100%"))

    #各タブのタイトルを設定
    top_tab.children = list(top_tab_dict.values())
    for i,title in enumerate(top_tab_dict.keys()):
      top_tab.set_title(i, title)
    
    self.layout = \
    widgets.HBox(layout=Layout(width="100%", height="100%"), children=[
      top_tab
    ])
    return self.layout

#Extraタブのインスタンスを生成
extraTab = ExtraTabView()


In [ ]:
#@title 全体レイアウト

# トップメニュータブと各ペインを設定
top_tab_dict = {"setup":setupPane.set_layout(), "txt2img":txt2imgPane.set_layout(), "img2img":img2imgPane.set_layout(), "inpaint":inpaintPane.set_layout(), "PNG Info":pnginfoPane.set_layout(), "Extras":extraTab.set_layout(), }
sharedBackEnd.setupPane = setupPane
sharedBackEnd.txt2imgPane = txt2imgPane
sharedBackEnd.img2imgPane = img2imgPane
sharedBackEnd.inpaintPane = inpaintPane
sharedBackEnd.pnginfoPane = pnginfoPane


top_tab = widgets.Tab(layout = Layout(width="100%",height="100%"))
top_tab.children = list(top_tab_dict.values())
for i,title in enumerate(top_tab_dict.keys()):
  top_tab.set_title(i, title)

# メインウィンドウを生成
main_window = widgets.Box([top_tab], layout = Layout(width="100%",height="100%"))

#3. 起動

In [ ]:
#@title GUIを起動。「出力を全画面表示」を推奨
main_window